# Fifa practice

We want to predict the *value* of the players using the information of the player.

## Import libraries

First of all we import the libraries we're gonna use.

In [1]:
import os

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.preprocessing import MinMaxScaler

import pandas as pd
import numpy as np

## Read the data

To read the data we're gonna use ``pandas`` and an utility from the library
``os``.


In [2]:
df = pd.read_csv(os.path.join("..", "in", "fifa.csv"))
df.head()

Unnamed: 0      ID               Name  Age  \
0           0  158023           L. Messi   31   
1           1   20801  Cristiano Ronaldo   33   
2           2  190871          Neymar Jr   26   
3           3  193080             De Gea   27   
4           4  192985       K. De Bruyne   27   

                                            Photo Nationality  \
0  https://cdn.sofifa.org/players/4/19/158023.png   Argentina   
1   https://cdn.sofifa.org/players/4/19/20801.png    Portugal   
2  https://cdn.sofifa.org/players/4/19/190871.png      Brazil   
3  https://cdn.sofifa.org/players/4/19/193080.png       Spain   
4  https://cdn.sofifa.org/players/4/19/192985.png     Belgium   

                                  Flag  Overall  Potential  \
0  https://cdn.sofifa.org/flags/52.png       94         94   
1  https://cdn.sofifa.org/flags/38.png       94         94   
2  https://cdn.sofifa.org/flags/54.png       92         93   
3  https://cdn.sofifa.org/flags/45.png       91         93   
4   https://cdn.sofifa.org/flags/7.png       91         92   

                  Club  ... Composure Marking StandingTackle  SlidingTackle  \
0         FC Barcelona  ...      96.0    33.0           28.0           26.0   
1             Juventus  ...      95.0    28.0           31.0           23.0   
2  Paris Saint-Germain  ...      94.0    27.0           24.0           33.0   
3    Manchester United  ...      68.0    15.0           21.0           13.0   
4      Manchester City  ...      88.0    68.0           58.0           51.0   

  GKDiving  GKHandling  GKKicking  GKPositioning GKReflexes Release Clause  
0      6.0        11.0       15.0           14.0        8.0        €226.5M  
1      7.0        11.0       15.0           14.0       11.0        €127.1M  
2      9.0         9.0       15.0           15.0       11.0        €228.1M  
3     90.0        85.0       87.0           88.0       94.0        €138.6M  
4     15.0        13.0        5.0           10.0       13.0        €196.4M  

[5 rows x 89 columns]

# Análisis de datos

Expandimos las columnas para que se muestren todas las columnas del data set.

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df.head()

Unnamed: 0      ID               Name  Age  \
0           0  158023           L. Messi   31   
1           1   20801  Cristiano Ronaldo   33   
2           2  190871          Neymar Jr   26   
3           3  193080             De Gea   27   
4           4  192985       K. De Bruyne   27   

                                            Photo Nationality  \
0  https://cdn.sofifa.org/players/4/19/158023.png   Argentina   
1   https://cdn.sofifa.org/players/4/19/20801.png    Portugal   
2  https://cdn.sofifa.org/players/4/19/190871.png      Brazil   
3  https://cdn.sofifa.org/players/4/19/193080.png       Spain   
4  https://cdn.sofifa.org/players/4/19/192985.png     Belgium   

                                  Flag  Overall  Potential  \
0  https://cdn.sofifa.org/flags/52.png       94         94   
1  https://cdn.sofifa.org/flags/38.png       94         94   
2  https://cdn.sofifa.org/flags/54.png       92         93   
3  https://cdn.sofifa.org/flags/45.png       91         93   
4   https://cdn.sofifa.org/flags/7.png       91         92   

                  Club                                     Club Logo    Value  \
0         FC Barcelona  https://cdn.sofifa.org/teams/2/light/241.png  €110.5M   
1             Juventus   https://cdn.sofifa.org/teams/2/light/45.png     €77M   
2  Paris Saint-Germain   https://cdn.sofifa.org/teams/2/light/73.png  €118.5M   
3    Manchester United   https://cdn.sofifa.org/teams/2/light/11.png     €72M   
4      Manchester City   https://cdn.sofifa.org/teams/2/light/10.png    €102M   

    Wage  Special Preferred Foot  International Reputation  Weak Foot  \
0  €565K     2202           Left                       5.0        4.0   
1  €405K     2228          Right                       5.0        4.0   
2  €290K     2143          Right                       5.0        5.0   
3  €260K     1471          Right                       4.0        3.0   
4  €355K     2281          Right                       4.0        5.0   

   Skill Moves       Work Rate   Body Type Real Face Position  Jersey Number  \
0          4.0  Medium/ Medium       Messi       Yes       RF           10.0   
1          5.0       High/ Low  C. Ronaldo       Yes       ST            7.0   
2          5.0    High/ Medium      Neymar       Yes       LW           10.0   
3          1.0  Medium/ Medium        Lean       Yes       GK            1.0   
4          4.0      High/ High      Normal       Yes      RCM            7.0   

         Joined Loaned From Contract Valid Until Height  Weight    LS    ST  \
0   Jul 1, 2004         NaN                 2021    5'7  159lbs  88+2  88+2   
1  Jul 10, 2018         NaN                 2022    6'2  183lbs  91+3  91+3   
2   Aug 3, 2017         NaN                 2022    5'9  150lbs  84+3  84+3   
3   Jul 1, 2011         NaN                 2020    6'4  168lbs   NaN   NaN   
4  Aug 30, 2015         NaN                 2023   5'11  154lbs  82+3  82+3   

     RS    LW    LF    CF    RF    RW   LAM   CAM   RAM    LM   LCM    CM  \
0  88+2  92+2  93+2  93+2  93+2  92+2  93+2  93+2  93+2  91+2  84+2  84+2   
1  91+3  89+3  90+3  90+3  90+3  89+3  88+3  88+3  88+3  88+3  81+3  81+3   
2  84+3  89+3  89+3  89+3  89+3  89+3  89+3  89+3  89+3  88+3  81+3  81+3   
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4  82+3  87+3  87+3  87+3  87+3  87+3  88+3  88+3  88+3  88+3  87+3  87+3   

    RCM    RM   LWB   LDM   CDM   RDM   RWB    LB   LCB    CB   RCB    RB  \
0  84+2  91+2  64+2  61+2  61+2  61+2  64+2  59+2  47+2  47+2  47+2  59+2   
1  81+3  88+3  65+3  61+3  61+3  61+3  65+3  61+3  53+3  53+3  53+3  61+3   
2  81+3  88+3  65+3  60+3  60+3  60+3  65+3  60+3  47+3  47+3  47+3  60+3   
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4  87+3  88+3  77+3  77+3  77+3  77+3  77+3  73+3  66+3  66+3  66+3  73+3   

   Crossing  Finishing  HeadingAccuracy  ShortPassing  Volleys  Dribbling  \
0      84.0       95.0             70.0          90.0     8

# Eliminamos todas las columnas que no aportan información relevante.

Las columnas eliminadas con esta sentencia son las que se ve fácilmente que no aportan información relevenate si entendemos la definición de los campos y aplicamos la lógica:

<b>'Name'</b>: el nombre no tiene ninguna relación con Value. Al algoritmo no le sirve de nada saber que un jugador con el nombre "Messi" es caro porque cualquier otro jugador con cualquier nombre también puede ser caro y aunque hubiera otro jugador llamado "Messi" en el FIFA no querría decir que por tener el mismo nombre sea igual de bueno que "Messi" y tenga el mismo Value.
    
<b>'Photo'</b>: la foto de un jugador no tiene relación con el Value. Esto es bastante evidente, una foto es simplemente el aspecto del jugador en un papel u otro material. ¿Además como trataría el algoritmo este campo?¿ Si le asignamos diferentes numeros a las fotos de los jugadores, de qué le sirve al algoritmo? No tiene sentido utilizarlo.

<b>'Flag'</b>:  ocurre lo mismo que con 'Photo'.
    
<b>'Club Logo'</b>: ocurre lo mismo que con 'Photo y 'Flag'.
    
 <b>'Body Type'</b>: esto indica el tipo de cuerpo del jugador en cuanto a forma (músculos, altura, etc). No es importante ya
              que podemos ver jugadores caros y baratos distribuidos en los diferentes BodyTypes. Los bodyType especí-
              ficos de jugadores como bodyType_Neymar tampoco sirven porque solo los tiene un único jugador, no sirven
              para extrapolar.
    
 <b>'Weak Foot'</b>: esto indica cuánto controla un jugador su pierna NO dominante. Es un factor que no influye prácticamente
              nada en cómo de bueno es un jugador y por tanto en su Value. Un jugador puede tener menos control de su pierna menos domiannte pero ser mucho mejor jugador en general que otro jugador que es ambidiestro. Por otra parte muchos porteros tienen baja puntuación en este campo y aun así son porteros muy caros.
                  
 <b>'Jersey Number'</b>: el número de la camiseta de un jugador no es importante porque es simplemente una manera de                             identificar al jugador dentro del equipo. Es cierto que podemos notar cierta popularidad de ciertos
                  números para determinadas posiciones, como 7 y 10 para delantero, pero esto no sirve, puede haber 
                  delanteros caros y delanteros baratos igualmente.
    
 <b>'Joined'</b>: este campo indica la fecha en que un jugador se unió a algo, probablemente al Club al que pertenece actualmente. Una fecha de este tipo no aporta información importante para predecir el Value de un jugador ya que el tiempo que hace que un jugador se ha unido a un club, es de lógica que no tiene que ver con la calidad del jugador. Que un jugador lleve más tiempo en un club que otro no tiene por qué significar que sea mejor. Por ejemplo cuando Cristiano se unió al Madrid ya erá un jugador más caro que otros que llevaban en el club por ejemplo 5 años. El mes, dia o año en que se haya unido un jugador tampoco tiene que ver con la calidad de este. Todo sobre este campo es irrelevante.

 <b>'Preferred Foot'</b>: este campo indica la pierna dominante de un jugador. No es importante para el objetivo de la práctica, ya que sabemos que hay tanto jugadores caros como baratos que son diestros,  y jugadores caros y baratos que son zurdos. Que un jugador fuera ambidiestro tampoco sería importante ya que aunque ser ambidiestro en general es mejor que dominar solo una pierna, eso no le asegura a un jugador ser tan bueno como un jugador caro cualquiera que es diestro o zurdo.
    
<b>'Contract Valid Until'</b>: este dato es irrelevante porque consiste en la fecha de vigencia del contrato de un  jugador. El hecho de que el contrato de un jugador termine antes o después no                                          tiene nada que ver con la calidad del jugador.
    
<b>'Height'</b>: la altura de un jugador no determina su Value. Tenemos jugadores caros de baja estatura como Messi, y jugadores altos caros como Cristiano Ronaldo, Zlatan Ibrahimović, etc. Esto es por que ser alto o bajo tiene cada uno sus respectivas ventajas (Por ejemplo un jugador bajo quizás es más ágil  mientras que un jugador alto tal vez le es más facil ser más rapido corriendo, etc) pero determinan directamente cómo de bueno es un jugador en general.
               
<b>'Weight'</b>: el peso de un jugador tampoco es importante porque aunque de primeras podríamos pensar que un jugador con mas peso significa que "está menos en forma" que otro con menos peso, recordemos que gran parte del peso de una persona es a causa de los huesos y de los músculos, no solo la grasa. Por lo tanto un jugador alto tendrá más peso que cualquier otro jugador mucho más bajo pero no significa que el más bajo sea un mejor jugador que el más alto. 
    
 <b>'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes'</b>: todos estos campos son propiedades de los porteros. Estos campos serían útiles seguramente si estuvieramos estudiando un dataFrame que solo contiene porteros. Pero dado que en el caso de esta práctica los porteros que hay representan solo una parte muy pequeña del total de jugadores, estos campos no sirven para deducir el Value de los jugadores en general.


In [5]:
df = df.iloc[:,2:]
df.drop(['Name','Photo','Flag','Club Logo','Body Type','Weak Foot','Jersey Number','Joined', 'Preferred Foot','Contract Valid Until','Height','Weight','GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes'], axis=1, inplace=True)
df.head()

Age Nationality  Overall  Potential                 Club    Value   Wage  \
0   31   Argentina       94         94         FC Barcelona  €110.5M  €565K   
1   33    Portugal       94         94             Juventus     €77M  €405K   
2   26      Brazil       92         93  Paris Saint-Germain  €118.5M  €290K   
3   27       Spain       91         93    Manchester United     €72M  €260K   
4   27     Belgium       91         92      Manchester City    €102M  €355K   

   Special  International Reputation  Skill Moves       Work Rate Real Face  \
0     2202                       5.0          4.0  Medium/ Medium       Yes   
1     2228                       5.0          5.0       High/ Low       Yes   
2     2143                       5.0          5.0    High/ Medium       Yes   
3     1471                       4.0          1.0  Medium/ Medium       Yes   
4     2281                       4.0          4.0      High/ High       Yes   

  Position Loaned From    LS    ST    RS    LW    LF    CF    RF    RW   LAM  \
0       RF         NaN  88+2  88+2  88+2  92+2  93+2  93+2  93+2  92+2  93+2   
1       ST         NaN  91+3  91+3  91+3  89+3  90+3  90+3  90+3  89+3  88+3   
2       LW         NaN  84+3  84+3  84+3  89+3  89+3  89+3  89+3  89+3  89+3   
3       GK         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4      RCM         NaN  82+3  82+3  82+3  87+3  87+3  87+3  87+3  87+3  88+3   

    CAM   RAM    LM   LCM    CM   RCM    RM   LWB   LDM   CDM   RDM   RWB  \
0  93+2  93+2  91+2  84+2  84+2  84+2  91+2  64+2  61+2  61+2  61+2  64+2   
1  88+3  88+3  88+3  81+3  81+3  81+3  88+3  65+3  61+3  61+3  61+3  65+3   
2  89+3  89+3  88+3  81+3  81+3  81+3  88+3  65+3  60+3  60+3  60+3  65+3   
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4  88+3  88+3  88+3  87+3  87+3  87+3  88+3  77+3  77+3  77+3  77+3  77+3   

     LB   LCB    CB   RCB    RB  Crossing  Finishing  HeadingAccuracy  \
0  59+2  47+2  47+2  47+2  59+2      84.0       95.0             70.0   
1  61+3  53+3  53+3  53+3  61+3      84.0       94.0             89.0   
2  60+3  47+3  47+3  47+3  60+3      79.0       87.0             62.0   
3   NaN   NaN   NaN   NaN   NaN      17.0       13.0             21.0   
4  73+3  66+3  66+3  66+3  73+3      93.0       82.0             55.0   

   ShortPassing  Volleys  Dribbling  Curve  FKAccuracy  LongPassing  \
0          90.0     86.0       97.0   93.0        94.0         87.0   
1          81.0     87.0       88.0   81.0        76.0         77.0   
2          84.0     84.0       96.0   88.0        87.0         78.0   
3          50.0     13.0       18.0   21.0        19.0         51.0   
4          92.0     82.0       86.0   85.0        83.0         91.0   

   BallControl  Acceleration  SprintSpeed  Agility  Reactions  Balance  \
0         96.0          91.0         86.0     91.0       95.0     95.0   
1         94.0          89.0         91.0     87.0       96.0     70.0   
2         95.0          94.0         90.0     96.0       94.0     84.0   
3         42.0          57.0         58.0     60.0       90.0     43.0   
4         91.0          78.0         76.0     79.0       91.0     77.0   

   ShotPower  Jumping  Stamina  Strength  LongShots  Aggression  \
0       85.0     68.0     72.0      59.0       94.0        48.0   
1       95.0     95.0     88.0      79.0       93.0        63.0   
2       80.0     61.0     81.0      49.0       82.0        56.0   
3       31.0     67.0     43.0      64.0       12.0        38.0   
4       91.0     63.0     90.0      75.0       91.0        76.0   

   Interceptions  Positioning  Vision  Penalties  Composure  Marking  \
0           22.0         94.0    94.0       75.0       96.0     33.0   
1           29.0         95.0    82.0       85.0       95.0     28.0   
2           36.0         89.0    87.0       81.0       94.0     27.0   
3           30.0         12.0    68.0       40.0       68.0     15.0   
4           61.0         87.0    94

Eliminamos los NaNs del data set

In [6]:
df.columns[df.isna().any()].tolist()

['Club',
 'International Reputation',
 'Skill Moves',
 'Work Rate',
 'Real Face',
 'Position',
 'Loaned From',
 'LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM',
 'CAM',
 'RAM',
 'LM',
 'LCM',
 'CM',
 'RCM',
 'RM',
 'LWB',
 'LDM',
 'CDM',
 'RDM',
 'RWB',
 'LB',
 'LCB',
 'CB',
 'RCB',
 'RB',
 'Crossing',
 'Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Reactions',
 'Balance',
 'ShotPower',
 'Jumping',
 'Stamina',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Marking',
 'StandingTackle',
 'SlidingTackle',
 'Release Clause']

In [7]:
df[df.isna().any(axis=1)]

Age Nationality  Overall  Potential                 Club    Value  \
0       31   Argentina       94         94         FC Barcelona  €110.5M   
1       33    Portugal       94         94             Juventus     €77M   
2       26      Brazil       92         93  Paris Saint-Germain  €118.5M   
3       27       Spain       91         93    Manchester United     €72M   
4       27     Belgium       91         92      Manchester City    €102M   
...    ...         ...      ...        ...                  ...      ...   
18202   19     England       47         65      Crewe Alexandra     €60K   
18203   19      Sweden       47         63       Trelleborgs FF     €60K   
18204   16     England       47         67     Cambridge United     €60K   
18205   17     England       47         66      Tranmere Rovers     €60K   
18206   16     England       46         66      Tranmere Rovers     €60K   

        Wage  Special  International Reputation  Skill Moves       Work Rate  \
0      €565K     2202                       5.0          4.0  Medium/ Medium   
1      €405K     2228                       5.0          5.0       High/ Low   
2      €290K     2143                       5.0          5.0    High/ Medium   
3      €260K     1471                       4.0          1.0  Medium/ Medium   
4      €355K     2281                       4.0          4.0      High/ High   
...      ...      ...                       ...          ...             ...   
18202    €1K     1307                       1.0          2.0  Medium/ Medium   
18203    €1K     1098                       1.0          2.0  Medium/ Medium   
18204    €1K     1189                       1.0          2.0  Medium/ Medium   
18205    €1K     1228                       1.0          2.0  Medium/ Medium   
18206    €1K     1321                       1.0          2.0  Medium/ Medium   

      Real Face Position Loaned From    LS    ST    RS    LW    LF    CF  \
0           Yes       RF         NaN  88+2  88+2  88+2  92+2  93+2  93+2   
1           Yes       ST         NaN  91+3  91+3  91+3  89+3  90+3  90+3   
2           Yes       LW         NaN  84+3  84+3  84+3  89+3  89+3  89+3   
3           Yes       GK         NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4           Yes      RCM         NaN  82+3  82+3  82+3  87+3  87+3  87+3   
...         ...      ...         ...   ...   ...   ...   ...   ...   ...   
18202        No       CM         NaN  42+2  42+2  42+2  44+2  44+2  44+2   
18203        No       ST         NaN  45+2  45+2  45+2  39+2  42+2  42+2   
18204        No       ST         NaN  45+2  45+2  45+2  45+2  46+2  46+2   
18205        No       RW         NaN  47+2  47+2  47+2  47+2  46+2  46+2   
18206        No       CM         NaN  43+2  43+2  43+2  45+2  44+2  44+2   

         RF    RW   LAM   CAM   RAM    LM   LCM    CM   RCM    RM   LWB   LDM  \
0      93+2  92+2  93+2  93+2  93+2  91+2  84+2  84+2  84+2  91+2  64+2  61+2   
1      90+3  89+3  88+3  88+3  88+3  88+3  81+3  81+3  81+3  88+3  65+3  61+3   
2      89+3  89+3  89+3  89+3  89+3  88+3  81+3  81+3  81+3  88+3  65+3  60+3   
3       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4      87+3  87+3  88+3  88+3  88+3  88+3  87+3  87+3  87+3  88+3  77+3  77+3   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
18202  44+2  44+2  45+2  45+2  45+2  44+2  45+2  45+2  45+2  44+2  44+2  45+2   
18203  42+2  39+2  40+2  40+2  40+2  38+2  35+2  35+2  35+2  38+2  30+2  31+2   
18204  46+2  45+2  44+2  44+2  44+2  44+2  38+2  38+2  38+2  44+2  34+2  30+2   
18205  46+2  47+2  45+2  45+2  45+2  46+2  39+2  39+2  39+2  46+2  36+2  32+2   
18206  44+2  45+2  45+2  45+2  45+2  46+2  45+2  45+2  45+2  46+2  46+2  46+2   

        CDM   RDM   RWB    LB   LCB    CB   RCB    RB  Crossing  Finishing  \
0      61+2  61+2  64+2  59+2  47+2  47+2  47+2  59+2      84.0       95.0   
1      61+3  61+3  65+3  61+3  53+3  53+3  53+3  61+3      84.0       94.0   
2      60+3  

Vemos que casi todos los jugadores tienen NaN en alguna columna

Quitamos la columna Loaned From porque siempre tiene NaN como valor

In [8]:
df.drop('Loaned From', axis=1, inplace=True)

In [9]:
df.columns[df.isna().any()].tolist()

['Club',
 'International Reputation',
 'Skill Moves',
 'Work Rate',
 'Real Face',
 'Position',
 'LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM',
 'CAM',
 'RAM',
 'LM',
 'LCM',
 'CM',
 'RCM',
 'RM',
 'LWB',
 'LDM',
 'CDM',
 'RDM',
 'RWB',
 'LB',
 'LCB',
 'CB',
 'RCB',
 'RB',
 'Crossing',
 'Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Reactions',
 'Balance',
 'ShotPower',
 'Jumping',
 'Stamina',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Marking',
 'StandingTackle',
 'SlidingTackle',
 'Release Clause']

Ahora vamos a analizar el resto de NaNs.

Por ejemplo podemos obtener las entradas que tienen NaN en la columna Volleys:

In [10]:
df.loc[df.Volleys != df.Volleys]

Age       Nationality  Overall  Potential  \
13236   33          Scotland       62         62   
13237   29         Nicaragua       62         62   
13238   35           England       62         62   
13239   20             Italy       62         72   
13240   24           England       62         66   
13241   21            Poland       62         72   
13242   23         Argentina       62         70   
13243   19           England       62         78   
13244   30           Denmark       62         62   
13245   21            Brazil       62         69   
13246   22            France       62         68   
13247   28           Austria       62         62   
13248   24  Northern Ireland       62         69   
13249   27          China PR       62         62   
13250   29          Paraguay       62         62   
13251   34           Belgium       62         62   
13252   22           Albania       62         70   
13253   31          Scotland       62         62   
13254   17           Germany       62         82   
13255   25           Germany       62         65   
13256   26           England       62         62   
13257   25    Korea Republic       62         65   
13258   23      Saudi Arabia       62         67   
13259   27             Chile       62         65   
13260   20             Spain       62         69   
13261   25      Saudi Arabia       62         64   
13262   24           Germany       62         66   
13263   25           Austria       62         66   
13264   29            France       62         62   
13265   28           England       62         62   
13266   20            Sweden       62         73   
13267   22          Colombia       62         70   
13268   37            Poland       62         62   
13269   26          Colombia       62         64   
13270   19            Brazil       62         77   
13271   26             Chile       62         63   
13272   22          Scotland       62         70   
13273   25           Estonia       62         67   
13274   27             Italy       62         62   
13275   23         Venezuela       62         70   
13276   26      Saudi Arabia       62         63   
13277   26             Chile       62         62   
13278   17       Netherlands       62         79   
13279   22             Italy       62         69   
13280   19            France       62         77   
13281   27    Korea Republic       62         62   
13282   25            Mexico       62         65   
13283   25          China PR       62         66   

                                Club  Value Wage  Special  \
13236                       Rochdale  €120K  €1K     1510   
13237                Boyacá Chicó FC  €300K  €1K     1532   
13238                   Notts County  €140K  €3K     1573   
13239                        Brescia  €425K  €1K     1610   
13240         Hamilton Academical FC  €400K  €1K     1481   
13241                  Śląsk Wrocław  €425K  €1K     1692   
13242         Club Atlético Aldosivi  €450K  €2K     1663   
13243                        Everton  €600K  €5K     1328   
13244                       Hobro IK  €230K  €2K     1244   
13245                   HJK Helsinki  €425K  €1K     1549   
13246                     AS Béziers  €425K  €2K     1494   
13247                 SV Mattersburg  €240K  €3K     1630   
13248                Tranmere Rovers  €375K  €2K     1461   
13249  Shanghai Greenland Shenhua FC  €250K  €3K     1636   
13250               Itagüí Leones FC  €300K  €1K     1454   
13251                      NAC Breda  €150K  €2K     1665   
13252                       Malmö FF  €375K  €1K     1587   
13253                Carlisle United  €200K  €2K     1535   
13254                  VfB Stuttgart  €550K  €2K     1418   
13255         Hamilton Academical FC  €325K  €1K     1693   
13256                      Dundee FC  €325K  €1K     1712   
13257        Suwon Samsung Bluewings  €375K  €2K     1536   
13258                       Al Wehda  €350K  €3K     16

Como vemos se trata de muy pocos jugadores. Además, los que tienen NaN en Volleys, parecen ser los que también tienen NaN en la mayoria de otros campos del DataFrame. Parece ser que se trata de unos pocos jugadores poco relevantes de los que no se conoce apenas información. Los podemos eliminar del DataFrame:

In [11]:
df = df.dropna(subset = ['Volleys'])
df

Age Nationality  Overall  Potential                 Club    Value  \
0       31   Argentina       94         94         FC Barcelona  €110.5M   
1       33    Portugal       94         94             Juventus     €77M   
2       26      Brazil       92         93  Paris Saint-Germain  €118.5M   
3       27       Spain       91         93    Manchester United     €72M   
4       27     Belgium       91         92      Manchester City    €102M   
...    ...         ...      ...        ...                  ...      ...   
18202   19     England       47         65      Crewe Alexandra     €60K   
18203   19      Sweden       47         63       Trelleborgs FF     €60K   
18204   16     England       47         67     Cambridge United     €60K   
18205   17     England       47         66      Tranmere Rovers     €60K   
18206   16     England       46         66      Tranmere Rovers     €60K   

        Wage  Special  International Reputation  Skill Moves       Work Rate  \
0      €565K     2202                       5.0          4.0  Medium/ Medium   
1      €405K     2228                       5.0          5.0       High/ Low   
2      €290K     2143                       5.0          5.0    High/ Medium   
3      €260K     1471                       4.0          1.0  Medium/ Medium   
4      €355K     2281                       4.0          4.0      High/ High   
...      ...      ...                       ...          ...             ...   
18202    €1K     1307                       1.0          2.0  Medium/ Medium   
18203    €1K     1098                       1.0          2.0  Medium/ Medium   
18204    €1K     1189                       1.0          2.0  Medium/ Medium   
18205    €1K     1228                       1.0          2.0  Medium/ Medium   
18206    €1K     1321                       1.0          2.0  Medium/ Medium   

      Real Face Position    LS    ST    RS    LW    LF    CF    RF    RW  \
0           Yes       RF  88+2  88+2  88+2  92+2  93+2  93+2  93+2  92+2   
1           Yes       ST  91+3  91+3  91+3  89+3  90+3  90+3  90+3  89+3   
2           Yes       LW  84+3  84+3  84+3  89+3  89+3  89+3  89+3  89+3   
3           Yes       GK   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4           Yes      RCM  82+3  82+3  82+3  87+3  87+3  87+3  87+3  87+3   
...         ...      ...   ...   ...   ...   ...   ...   ...   ...   ...   
18202        No       CM  42+2  42+2  42+2  44+2  44+2  44+2  44+2  44+2   
18203        No       ST  45+2  45+2  45+2  39+2  42+2  42+2  42+2  39+2   
18204        No       ST  45+2  45+2  45+2  45+2  46+2  46+2  46+2  45+2   
18205        No       RW  47+2  47+2  47+2  47+2  46+2  46+2  46+2  47+2   
18206        No       CM  43+2  43+2  43+2  45+2  44+2  44+2  44+2  45+2   

        LAM   CAM   RAM    LM   LCM    CM   RCM    RM   LWB   LDM   CDM   RDM  \
0      93+2  93+2  93+2  91+2  84+2  84+2  84+2  91+2  64+2  61+2  61+2  61+2   
1      88+3  88+3  88+3  88+3  81+3  81+3  81+3  88+3  65+3  61+3  61+3  61+3   
2      89+3  89+3  89+3  88+3  81+3  81+3  81+3  88+3  65+3  60+3  60+3  60+3   
3       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4      88+3  88+3  88+3  88+3  87+3  87+3  87+3  88+3  77+3  77+3  77+3  77+3   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
18202  45+2  45+2  45+2  44+2  45+2  45+2  45+2  44+2  44+2  45+2  45+2  45+2   
18203  40+2  40+2  40+2  38+2  35+2  35+2  35+2  38+2  30+2  31+2  31+2  31+2   
18204  44+2  44+2  44+2  44+2  38+2  38+2  38+2  44+2  34+2  30+2  30+2  30+2   
18205  45+2  45+2  45+2  46+2  39+2  39+2  39+2  46+2  36+2  32+2  32+2  32+2   
18206  45+2  45+2  45+2  46+2  45+2  45+2  45+2  46+2  46+2  46+2  46+2  46+2   

        RWB    LB   LCB    CB   RCB    RB  Crossing  Finishing  \
0      64+2  59+2  47+2  47+2  47+2  59+2      84.0       95.0   
1      65+3  61+3  53+3  53+3  53+3  61+3      84.0       94.0   
2      65+3  60+3  47+3  47+3  47+3  60+3      79

Veamos ahora como queda la lista de columnas que contienen algún NaN:

In [12]:
df.columns[df.isna().any()].tolist()

['Club',
 'Position',
 'LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM',
 'CAM',
 'RAM',
 'LM',
 'LCM',
 'CM',
 'RCM',
 'RM',
 'LWB',
 'LDM',
 'CDM',
 'RDM',
 'RWB',
 'LB',
 'LCB',
 'CB',
 'RCB',
 'RB',
 'Release Clause']

¿Qué jugadores tienen NaN en la columna Club?

In [13]:
df.loc[df.Club != df.Club]

Age Nationality  Overall  Potential Club Value Wage  Special  \
452     24   Argentina       80         85  NaN    €0   €0     2122   
538     33      Sweden       80         80  NaN    €0   €0     1797   
568     26      Russia       79         81  NaN    €0   €0     1217   
677     29      Russia       79         79  NaN    €0   €0     2038   
874     29      Russia       78         78  NaN    €0   €0     1810   
...    ...         ...      ...        ...  ...   ...  ...      ...   
17197   21       India       55         64  NaN    €0   €0      838   
17215   26     Finland       55         57  NaN    €0   €0     1366   
17339   23       India       54         63  NaN    €0   €0     1321   
17436   20       India       54         67  NaN    €0   €0     1270   
17539   21       India       53         62  NaN    €0   €0     1247   

       International Reputation  Skill Moves       Work Rate Real Face  \
452                         2.0          4.0  Medium/ Medium        No   
538                         2.0          2.0    High/ Medium        No   
568                         1.0          1.0  Medium/ Medium        No   
677                         2.0          3.0      High/ High        No   
874                         2.0          3.0    High/ Medium        No   
...                         ...          ...             ...       ...   
17197                       1.0          1.0  Medium/ Medium        No   
17215                       1.0          2.0    Medium/ High        No   
17339                       1.0          2.0     Medium/ Low        No   
17436                       1.0          2.0  Medium/ Medium        No   
17539                       1.0          2.0    High/ Medium        No   

      Position    LS    ST    RS    LW    LF    CF    RF    RW   LAM   CAM  \
452         CM  71+2  71+2  71+2  75+2  75+2  75+2  75+2  75+2  77+2  77+2   
538        LCB  62+2  62+2  62+2  56+2  58+2  58+2  58+2  56+2  58+2  58+2   
568         GK   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
677         RB  70+2  70+2  70+2  73+2  72+2  72+2  72+2  73+2  73+2  73+2   
874         ST  77+2  77+2  77+2  71+2  74+2  74+2  74+2  71+2  71+2  71+2   
...        ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
17197       GK   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
17215       RB  46+2  46+2  46+2  45+2  44+2  44+2  44+2  45+2  43+2  43+2   
17339      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
17436      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
17539      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

        RAM    LM   LCM    CM   RCM    RM   LWB   LDM   CDM   RDM   RWB    LB  \
452    77+2  76+2  79+2  79+2  79+2  76+2  75+2  77+2  77+2  77+2  75+2  74+2   
538    58+2  57+2  64+2  64+2  64+2  57+2  68+2  74+2  74+2  74+2  68+2  70+2   
568     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
677    73+2  75+2  74+2  74+2  74+2  75+2  78+2  75+2  75+2  75+2  78+2  78+2   
874    71+2  71+2  66+2  66+2  66+2  71+2  52+2  52+2  52+2  52+2  52+2  48+2   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
17197   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
17215  43+2  46+2  43+2  43+2  43+2  46+2  51+2  49+2  49+2  49+2  51+2  53+2   
17339   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
17436   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
17539   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

        LCB    CB   RCB    RB  Crossing  Finishing  HeadingAccuracy  \
452    72+2  72+2  72+2  74+2      76.0       55.0             60.0   
538    79+2  79+2  79+2  70+2      49.0       51.0             81.0   
568     NaN   NaN   NaN   NaN      16.0       14.0             17.0   
677    73+2  73+2  73+2  78+2      73.0       61.0             69.0   


Vemos que solo son 241, los eliminamos, ya que un número tan reducido no afectará al rendimiento del algoritmo:

In [14]:
df = df.dropna(subset = ['Club'])
df

Age Nationality  Overall  Potential                 Club    Value  \
0       31   Argentina       94         94         FC Barcelona  €110.5M   
1       33    Portugal       94         94             Juventus     €77M   
2       26      Brazil       92         93  Paris Saint-Germain  €118.5M   
3       27       Spain       91         93    Manchester United     €72M   
4       27     Belgium       91         92      Manchester City    €102M   
...    ...         ...      ...        ...                  ...      ...   
18202   19     England       47         65      Crewe Alexandra     €60K   
18203   19      Sweden       47         63       Trelleborgs FF     €60K   
18204   16     England       47         67     Cambridge United     €60K   
18205   17     England       47         66      Tranmere Rovers     €60K   
18206   16     England       46         66      Tranmere Rovers     €60K   

        Wage  Special  International Reputation  Skill Moves       Work Rate  \
0      €565K     2202                       5.0          4.0  Medium/ Medium   
1      €405K     2228                       5.0          5.0       High/ Low   
2      €290K     2143                       5.0          5.0    High/ Medium   
3      €260K     1471                       4.0          1.0  Medium/ Medium   
4      €355K     2281                       4.0          4.0      High/ High   
...      ...      ...                       ...          ...             ...   
18202    €1K     1307                       1.0          2.0  Medium/ Medium   
18203    €1K     1098                       1.0          2.0  Medium/ Medium   
18204    €1K     1189                       1.0          2.0  Medium/ Medium   
18205    €1K     1228                       1.0          2.0  Medium/ Medium   
18206    €1K     1321                       1.0          2.0  Medium/ Medium   

      Real Face Position    LS    ST    RS    LW    LF    CF    RF    RW  \
0           Yes       RF  88+2  88+2  88+2  92+2  93+2  93+2  93+2  92+2   
1           Yes       ST  91+3  91+3  91+3  89+3  90+3  90+3  90+3  89+3   
2           Yes       LW  84+3  84+3  84+3  89+3  89+3  89+3  89+3  89+3   
3           Yes       GK   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4           Yes      RCM  82+3  82+3  82+3  87+3  87+3  87+3  87+3  87+3   
...         ...      ...   ...   ...   ...   ...   ...   ...   ...   ...   
18202        No       CM  42+2  42+2  42+2  44+2  44+2  44+2  44+2  44+2   
18203        No       ST  45+2  45+2  45+2  39+2  42+2  42+2  42+2  39+2   
18204        No       ST  45+2  45+2  45+2  45+2  46+2  46+2  46+2  45+2   
18205        No       RW  47+2  47+2  47+2  47+2  46+2  46+2  46+2  47+2   
18206        No       CM  43+2  43+2  43+2  45+2  44+2  44+2  44+2  45+2   

        LAM   CAM   RAM    LM   LCM    CM   RCM    RM   LWB   LDM   CDM   RDM  \
0      93+2  93+2  93+2  91+2  84+2  84+2  84+2  91+2  64+2  61+2  61+2  61+2   
1      88+3  88+3  88+3  88+3  81+3  81+3  81+3  88+3  65+3  61+3  61+3  61+3   
2      89+3  89+3  89+3  88+3  81+3  81+3  81+3  88+3  65+3  60+3  60+3  60+3   
3       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4      88+3  88+3  88+3  88+3  87+3  87+3  87+3  88+3  77+3  77+3  77+3  77+3   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
18202  45+2  45+2  45+2  44+2  45+2  45+2  45+2  44+2  44+2  45+2  45+2  45+2   
18203  40+2  40+2  40+2  38+2  35+2  35+2  35+2  38+2  30+2  31+2  31+2  31+2   
18204  44+2  44+2  44+2  44+2  38+2  38+2  38+2  44+2  34+2  30+2  30+2  30+2   
18205  45+2  45+2  45+2  46+2  39+2  39+2  39+2  46+2  36+2  32+2  32+2  32+2   
18206  45+2  45+2  45+2  46+2  45+2  45+2  45+2  46+2  46+2  46+2  46+2  46+2   

        RWB    LB   LCB    CB   RCB    RB  Crossing  Finishing  \
0      64+2  59+2  47+2  47+2  47+2  59+2      84.0       95.0   
1      65+3  61+3  53+3  53+3  53+3  61+3      84.0       94.0   
2      65+3  60+3  47+3  47+3  47+3  60+3      79

¿Y ahora como queda la lista de columnas con NaN?

In [15]:
df.columns[df.isna().any()].tolist()

['LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM',
 'CAM',
 'RAM',
 'LM',
 'LCM',
 'CM',
 'RCM',
 'RM',
 'LWB',
 'LDM',
 'CDM',
 'RDM',
 'RWB',
 'LB',
 'LCB',
 'CB',
 'RCB',
 'RB',
 'Release Clause']

Ahora los únicos NaN que quedan son las posiciones de los jugadores que son porteros y algunos NaN de la columna 'Release Clause'. Vamos a ver cuántas entradas hay que tienen NaN en Release Clause:

In [16]:
df.loc[df["Release Clause"] != df["Release Clause"]]

Age Nationality  Overall  Potential                            Club  \
28      26    Colombia       88         89               FC Bayern München   
38      30   Argentina       88         88                           Milan   
91      29      Brazil       85         85  Guangzhou Evergrande Taobao FC   
166     24      Brazil       83         90  Guangzhou Evergrande Taobao FC   
176     24     Croatia       83         89                         Chelsea   
...    ...         ...      ...        ...                             ...   
17979   21    China PR       51         60             Guizhou Hengfeng FC   
18026   21    China PR       50         59             Guizhou Hengfeng FC   
18031   20    China PR       50         61                  Stabæk Fotball   
18056   19       Italy       50         65                          Ascoli   
18183   44     England       48         48                Cambridge United   

        Value   Wage  Special  International Reputation  Skill Moves  \
28     €69.5M  €315K     2171                       4.0          4.0   
38       €57M  €245K     1965                       4.0          3.0   
91       €37M  €235K     2245                       3.0          3.0   
166    €36.5M   €18K     2142                       2.0          4.0   
176      €35M  €165K     2115                       3.0          4.0   
...       ...    ...      ...                       ...          ...   
17979    €60K    €2K     1459                       1.0          2.0   
18026    €50K    €2K     1440                       1.0          2.0   
18031    €40K    €2K     1278                       1.0          2.0   
18056    €60K    €3K     1450                       1.0          2.0   
18183      €0    €1K      774                       1.0          1.0   

            Work Rate Real Face Position    LS    ST    RS    LW    LF    CF  \
28     Medium/ Medium       Yes      LAM  80+3  80+3  80+3  84+3  83+3  83+3   
38       High/ Medium       Yes       LS  85+3  85+3  85+3  81+3  83+3  83+3   
91         High/ High       Yes      LDM  81+3  81+3  81+3  79+3  81+3  81+3   
166      High/ Medium        No      CAM  80+2  80+2  80+2  81+2  81+2  81+2   
176        High/ High       Yes      LCM  71+3  71+3  71+3  79+3  78+3  78+3   
...               ...       ...      ...   ...   ...   ...   ...   ...   ...   
17979  Medium/ Medium        No       CM  49+2  49+2  49+2  50+2  50+2  50+2   
18026  Medium/ Medium        No       LM  44+2  44+2  44+2  47+2  46+2  46+2   
18031  Medium/ Medium        No       RB  37+2  37+2  37+2  42+2  40+2  40+2   
18056  Medium/ Medium        No       CM  44+2  44+2  44+2  47+2  45+2  45+2   
18183  Medium/ Medium        No       GK   NaN   NaN   NaN   NaN   NaN   NaN   

         RF    RW   LAM   CAM   RAM    LM   LCM    CM   RCM    RM   LWB   LDM  \
28     83+3  84+3  85+3  85+3  85+3  83+3  81+3  81+3  81+3  83+3  69+3  68+3   
38     83+3  81+3  80+3  80+3  80+3  78+3  71+3  71+3  71+3  78+3  55+3  53+3   
91     81+3  79+3  81+3  81+3  81+3  79+3  82+3  82+3  82+3  79+3  78+3  81+3   
166    81+2  81+2  81+2  81+2  81+2  80+2  79+2  79+2  79+2  80+2  69+2  69+2   
176    78+3  79+3  81+3  81+3  81+3  80+3  82+3  82+3  82+3  80+3  76+3  78+3   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
17979  50+2  50+2  50+2  50+2  50+2  51+2  50+2  50+2  50+2  51+2  50+2  50+2   
18026  46+2  47+2  48+2  48+2  48+2  49+2  48+2  48+2  48+2  49+2  50+2  50+2   
18031  40+2  42+2  39+2  39+2  39+2  43+2  38+2  38+2  38+2  43+2  48+2  43+2   
18056  45+2  47+2  47+2  47+2  47+2  49+2  48+2  48+2  48+2  49+2  52+2  51+2   
18183   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

        CDM   RDM   RWB    LB   LCB    CB   RCB    RB  Crossing  Finishing  \
28     68+3  68+3  69+3  65+3  58+3  58+3  58+3  65+3      90.0       83.0   
38     53+3  53+3  55+3  51+3  48+3  48+3  48+3  51+3      68.0       92.0   
91     81+3  81+3  78+3  78+3  80+3  

Como vemos son solo 1275 jugadores, por lo que podemos eliminarlos de la tabla sin que el funcionamiento del algoritmo se vea afectado:

In [17]:
df = df.dropna(subset = ['Release Clause'])
df

Age Nationality  Overall  Potential                 Club    Value  \
0       31   Argentina       94         94         FC Barcelona  €110.5M   
1       33    Portugal       94         94             Juventus     €77M   
2       26      Brazil       92         93  Paris Saint-Germain  €118.5M   
3       27       Spain       91         93    Manchester United     €72M   
4       27     Belgium       91         92      Manchester City    €102M   
...    ...         ...      ...        ...                  ...      ...   
18202   19     England       47         65      Crewe Alexandra     €60K   
18203   19      Sweden       47         63       Trelleborgs FF     €60K   
18204   16     England       47         67     Cambridge United     €60K   
18205   17     England       47         66      Tranmere Rovers     €60K   
18206   16     England       46         66      Tranmere Rovers     €60K   

        Wage  Special  International Reputation  Skill Moves       Work Rate  \
0      €565K     2202                       5.0          4.0  Medium/ Medium   
1      €405K     2228                       5.0          5.0       High/ Low   
2      €290K     2143                       5.0          5.0    High/ Medium   
3      €260K     1471                       4.0          1.0  Medium/ Medium   
4      €355K     2281                       4.0          4.0      High/ High   
...      ...      ...                       ...          ...             ...   
18202    €1K     1307                       1.0          2.0  Medium/ Medium   
18203    €1K     1098                       1.0          2.0  Medium/ Medium   
18204    €1K     1189                       1.0          2.0  Medium/ Medium   
18205    €1K     1228                       1.0          2.0  Medium/ Medium   
18206    €1K     1321                       1.0          2.0  Medium/ Medium   

      Real Face Position    LS    ST    RS    LW    LF    CF    RF    RW  \
0           Yes       RF  88+2  88+2  88+2  92+2  93+2  93+2  93+2  92+2   
1           Yes       ST  91+3  91+3  91+3  89+3  90+3  90+3  90+3  89+3   
2           Yes       LW  84+3  84+3  84+3  89+3  89+3  89+3  89+3  89+3   
3           Yes       GK   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4           Yes      RCM  82+3  82+3  82+3  87+3  87+3  87+3  87+3  87+3   
...         ...      ...   ...   ...   ...   ...   ...   ...   ...   ...   
18202        No       CM  42+2  42+2  42+2  44+2  44+2  44+2  44+2  44+2   
18203        No       ST  45+2  45+2  45+2  39+2  42+2  42+2  42+2  39+2   
18204        No       ST  45+2  45+2  45+2  45+2  46+2  46+2  46+2  45+2   
18205        No       RW  47+2  47+2  47+2  47+2  46+2  46+2  46+2  47+2   
18206        No       CM  43+2  43+2  43+2  45+2  44+2  44+2  44+2  45+2   

        LAM   CAM   RAM    LM   LCM    CM   RCM    RM   LWB   LDM   CDM   RDM  \
0      93+2  93+2  93+2  91+2  84+2  84+2  84+2  91+2  64+2  61+2  61+2  61+2   
1      88+3  88+3  88+3  88+3  81+3  81+3  81+3  88+3  65+3  61+3  61+3  61+3   
2      89+3  89+3  89+3  88+3  81+3  81+3  81+3  88+3  65+3  60+3  60+3  60+3   
3       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4      88+3  88+3  88+3  88+3  87+3  87+3  87+3  88+3  77+3  77+3  77+3  77+3   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
18202  45+2  45+2  45+2  44+2  45+2  45+2  45+2  44+2  44+2  45+2  45+2  45+2   
18203  40+2  40+2  40+2  38+2  35+2  35+2  35+2  38+2  30+2  31+2  31+2  31+2   
18204  44+2  44+2  44+2  44+2  38+2  38+2  38+2  44+2  34+2  30+2  30+2  30+2   
18205  45+2  45+2  45+2  46+2  39+2  39+2  39+2  46+2  36+2  32+2  32+2  32+2   
18206  45+2  45+2  45+2  46+2  45+2  45+2  45+2  46+2  46+2  46+2  46+2  46+2   

        RWB    LB   LCB    CB   RCB    RB  Crossing  Finishing  \
0      64+2  59+2  47+2  47+2  47+2  59+2      84.0       95.0   
1      65+3  61+3  53+3  53+3  53+3  61+3      84.0       94.0   
2      65+3  60+3  47+3  47+3  47+3  60+3      79

Ya solo quedan los NaN de los porteros:

In [18]:
df.columns[df.isna().any()].tolist()

['LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM',
 'CAM',
 'RAM',
 'LM',
 'LCM',
 'CM',
 'RCM',
 'RM',
 'LWB',
 'LDM',
 'CDM',
 'RDM',
 'RWB',
 'LB',
 'LCB',
 'CB',
 'RCB',
 'RB']

Como el algoritmo no puede tratar NaNs, vamos a asignar el mismo valor numérico a todas las posiciones de todos los porteros.

Asignamos el valor 0 a todas las posiciones de todos los porteros:

In [19]:
def cambiarNaNporCero(x):
    if x!=x: 
        return '0'
    else:
        return x

In [20]:
import copy

df_c = copy.deepcopy(df)
columnNames = ["LS","ST","RS","LW","LF","CF","RF","RW","LAM","CAM","RAM","LM","LCM","CM","RCM","RM","LWB","LDM","CDM","RDM","RWB","LB","LCB","CB","RCB","RB"]

for columnName in columnNames:
    df_c.loc[:,columnName] = df_c.loc[:,columnName].apply(cambiarNaNporCero).copy()
    
df = df_c


In [21]:
df

Age Nationality  Overall  Potential                 Club    Value  \
0       31   Argentina       94         94         FC Barcelona  €110.5M   
1       33    Portugal       94         94             Juventus     €77M   
2       26      Brazil       92         93  Paris Saint-Germain  €118.5M   
3       27       Spain       91         93    Manchester United     €72M   
4       27     Belgium       91         92      Manchester City    €102M   
...    ...         ...      ...        ...                  ...      ...   
18202   19     England       47         65      Crewe Alexandra     €60K   
18203   19      Sweden       47         63       Trelleborgs FF     €60K   
18204   16     England       47         67     Cambridge United     €60K   
18205   17     England       47         66      Tranmere Rovers     €60K   
18206   16     England       46         66      Tranmere Rovers     €60K   

        Wage  Special  International Reputation  Skill Moves       Work Rate  \
0      €565K     2202                       5.0          4.0  Medium/ Medium   
1      €405K     2228                       5.0          5.0       High/ Low   
2      €290K     2143                       5.0          5.0    High/ Medium   
3      €260K     1471                       4.0          1.0  Medium/ Medium   
4      €355K     2281                       4.0          4.0      High/ High   
...      ...      ...                       ...          ...             ...   
18202    €1K     1307                       1.0          2.0  Medium/ Medium   
18203    €1K     1098                       1.0          2.0  Medium/ Medium   
18204    €1K     1189                       1.0          2.0  Medium/ Medium   
18205    €1K     1228                       1.0          2.0  Medium/ Medium   
18206    €1K     1321                       1.0          2.0  Medium/ Medium   

      Real Face Position    LS    ST    RS    LW    LF    CF    RF    RW  \
0           Yes       RF  88+2  88+2  88+2  92+2  93+2  93+2  93+2  92+2   
1           Yes       ST  91+3  91+3  91+3  89+3  90+3  90+3  90+3  89+3   
2           Yes       LW  84+3  84+3  84+3  89+3  89+3  89+3  89+3  89+3   
3           Yes       GK     0     0     0     0     0     0     0     0   
4           Yes      RCM  82+3  82+3  82+3  87+3  87+3  87+3  87+3  87+3   
...         ...      ...   ...   ...   ...   ...   ...   ...   ...   ...   
18202        No       CM  42+2  42+2  42+2  44+2  44+2  44+2  44+2  44+2   
18203        No       ST  45+2  45+2  45+2  39+2  42+2  42+2  42+2  39+2   
18204        No       ST  45+2  45+2  45+2  45+2  46+2  46+2  46+2  45+2   
18205        No       RW  47+2  47+2  47+2  47+2  46+2  46+2  46+2  47+2   
18206        No       CM  43+2  43+2  43+2  45+2  44+2  44+2  44+2  45+2   

        LAM   CAM   RAM    LM   LCM    CM   RCM    RM   LWB   LDM   CDM   RDM  \
0      93+2  93+2  93+2  91+2  84+2  84+2  84+2  91+2  64+2  61+2  61+2  61+2   
1      88+3  88+3  88+3  88+3  81+3  81+3  81+3  88+3  65+3  61+3  61+3  61+3   
2      89+3  89+3  89+3  88+3  81+3  81+3  81+3  88+3  65+3  60+3  60+3  60+3   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4      88+3  88+3  88+3  88+3  87+3  87+3  87+3  88+3  77+3  77+3  77+3  77+3   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
18202  45+2  45+2  45+2  44+2  45+2  45+2  45+2  44+2  44+2  45+2  45+2  45+2   
18203  40+2  40+2  40+2  38+2  35+2  35+2  35+2  38+2  30+2  31+2  31+2  31+2   
18204  44+2  44+2  44+2  44+2  38+2  38+2  38+2  44+2  34+2  30+2  30+2  30+2   
18205  45+2  45+2  45+2  46+2  39+2  39+2  39+2  46+2  36+2  32+2  32+2  32+2   
18206  45+2  45+2  45+2  46+2  45+2  45+2  45+2  46+2  46+2  46+2  46+2  46+2   

        RWB    LB   LCB    CB   RCB    RB  Crossing  Finishing  \
0      64+2  59+2  47+2  47+2  47+2  59+2      84.0       95.0   
1      65+3  61+3  53+3  53+3  53+3  61+3      84.0       94.0   
2      65+3  60+3  47+3  47+3  47+3  60+3      79

Como vemos tras quitar todos los NaN de todas las posiciones de todos los porteros, si pedimos que nos enseñe las filas
que contienen algún NaN, devuelve el dataFrame vacio. Los porteros eran el último problema a solucionar en cuanto a NaNs:

In [22]:
df.columns[df.isna().any()].tolist()

[]

Si ahora ejecutamos la sentencia para obtener la lista de NaNs restantes, también la devuelve vacía.

# Vamos a modificar las columnas que que contienen valores numéricos mezclados con carácteres para que el algoritmo pueda tratar los valores de estas

- 'Value'
- 'Wage'
- 'Release Clause'
- 'Posiciones de los jugadores'

In [23]:
def value_to_float(x):
    """
    From K and M to float.
    
    """
    x = x.replace('€', '')
    ret_val = 0.0
    
    if type(x) == float or type(x) == int:
        ret_val = x
    if 'K' in x:
        if len(x) > 1:
            ret_val = float(x.replace('K', ''))
        ret_val = ret_val *1000
    if 'M' in x:
        if len(x) > 1:
            ret_val = float(x.replace('M', ''))
        ret_val = ret_val * 1000000.0
    return ret_val

In [24]:
df.loc[:,'Value'] = df.loc[:,'Value'].apply(value_to_float)
df.head()

Age Nationality  Overall  Potential                 Club        Value  \
0   31   Argentina       94         94         FC Barcelona  110500000.0   
1   33    Portugal       94         94             Juventus   77000000.0   
2   26      Brazil       92         93  Paris Saint-Germain  118500000.0   
3   27       Spain       91         93    Manchester United   72000000.0   
4   27     Belgium       91         92      Manchester City  102000000.0   

    Wage  Special  International Reputation  Skill Moves       Work Rate  \
0  €565K     2202                       5.0          4.0  Medium/ Medium   
1  €405K     2228                       5.0          5.0       High/ Low   
2  €290K     2143                       5.0          5.0    High/ Medium   
3  €260K     1471                       4.0          1.0  Medium/ Medium   
4  €355K     2281                       4.0          4.0      High/ High   

  Real Face Position    LS    ST    RS    LW    LF    CF    RF    RW   LAM  \
0       Yes       RF  88+2  88+2  88+2  92+2  93+2  93+2  93+2  92+2  93+2   
1       Yes       ST  91+3  91+3  91+3  89+3  90+3  90+3  90+3  89+3  88+3   
2       Yes       LW  84+3  84+3  84+3  89+3  89+3  89+3  89+3  89+3  89+3   
3       Yes       GK     0     0     0     0     0     0     0     0     0   
4       Yes      RCM  82+3  82+3  82+3  87+3  87+3  87+3  87+3  87+3  88+3   

    CAM   RAM    LM   LCM    CM   RCM    RM   LWB   LDM   CDM   RDM   RWB  \
0  93+2  93+2  91+2  84+2  84+2  84+2  91+2  64+2  61+2  61+2  61+2  64+2   
1  88+3  88+3  88+3  81+3  81+3  81+3  88+3  65+3  61+3  61+3  61+3  65+3   
2  89+3  89+3  88+3  81+3  81+3  81+3  88+3  65+3  60+3  60+3  60+3  65+3   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4  88+3  88+3  88+3  87+3  87+3  87+3  88+3  77+3  77+3  77+3  77+3  77+3   

     LB   LCB    CB   RCB    RB  Crossing  Finishing  HeadingAccuracy  \
0  59+2  47+2  47+2  47+2  59+2      84.0       95.0             70.0   
1  61+3  53+3  53+3  53+3  61+3      84.0       94.0             89.0   
2  60+3  47+3  47+3  47+3  60+3      79.0       87.0             62.0   
3     0     0     0     0     0      17.0       13.0             21.0   
4  73+3  66+3  66+3  66+3  73+3      93.0       82.0             55.0   

   ShortPassing  Volleys  Dribbling  Curve  FKAccuracy  LongPassing  \
0          90.0     86.0       97.0   93.0        94.0         87.0   
1          81.0     87.0       88.0   81.0        76.0         77.0   
2          84.0     84.0       96.0   88.0        87.0         78.0   
3          50.0     13.0       18.0   21.0        19.0         51.0   
4          92.0     82.0       86.0   85.0        83.0         91.0   

   BallControl  Acceleration  SprintSpeed  Agility  Reactions  Balance  \
0         96.0          91.0         86.0     91.0       95.0     95.0   
1         94.0          89.0         91.0     87.0       96.0     70.0   
2         95.0          94.0         90.0     96.0       94.0     84.0   
3         42.0          57.0         58.0     60.0       90.0     43.0   
4         91.0          78.0         76.0     79.0       91.0     77.0   

   ShotPower  Jumping  Stamina  Strength  LongShots  Aggression  \
0       85.0     68.0     72.0      59.0       94.0        48.0   
1       95.0     95.0     88.0      79.0       93.0        63.0   
2       80.0     61.0     81.0      49.0       82.0        56.0   
3       31.0     67.0     43.0      64.0       12.0        38.0   
4       91.0     63.0     90.0      75.0       91.0        76.0   

   Interceptions  Positioning  Vision  Penalties  Composure  Marking  \
0           22.0         94.0    94.0       75.0       96.0     33.0   
1           29.0         95.0    82.0       85.0       95.0     28.0   
2           36.0         89.0    87.0       81.0       94.0     27.0   
3           30.0         12.0    68.0       40.0       68.0     15.0   
4           61.0         87.0    94.0       79.0       88.0     68.0   

   Standin

In [25]:
df.loc[:,'Wage'] = df.loc[:,'Wage'].apply(value_to_float)
df.head()

Age Nationality  Overall  Potential                 Club        Value  \
0   31   Argentina       94         94         FC Barcelona  110500000.0   
1   33    Portugal       94         94             Juventus   77000000.0   
2   26      Brazil       92         93  Paris Saint-Germain  118500000.0   
3   27       Spain       91         93    Manchester United   72000000.0   
4   27     Belgium       91         92      Manchester City  102000000.0   

       Wage  Special  International Reputation  Skill Moves       Work Rate  \
0  565000.0     2202                       5.0          4.0  Medium/ Medium   
1  405000.0     2228                       5.0          5.0       High/ Low   
2  290000.0     2143                       5.0          5.0    High/ Medium   
3  260000.0     1471                       4.0          1.0  Medium/ Medium   
4  355000.0     2281                       4.0          4.0      High/ High   

  Real Face Position    LS    ST    RS    LW    LF    CF    RF    RW   LAM  \
0       Yes       RF  88+2  88+2  88+2  92+2  93+2  93+2  93+2  92+2  93+2   
1       Yes       ST  91+3  91+3  91+3  89+3  90+3  90+3  90+3  89+3  88+3   
2       Yes       LW  84+3  84+3  84+3  89+3  89+3  89+3  89+3  89+3  89+3   
3       Yes       GK     0     0     0     0     0     0     0     0     0   
4       Yes      RCM  82+3  82+3  82+3  87+3  87+3  87+3  87+3  87+3  88+3   

    CAM   RAM    LM   LCM    CM   RCM    RM   LWB   LDM   CDM   RDM   RWB  \
0  93+2  93+2  91+2  84+2  84+2  84+2  91+2  64+2  61+2  61+2  61+2  64+2   
1  88+3  88+3  88+3  81+3  81+3  81+3  88+3  65+3  61+3  61+3  61+3  65+3   
2  89+3  89+3  88+3  81+3  81+3  81+3  88+3  65+3  60+3  60+3  60+3  65+3   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4  88+3  88+3  88+3  87+3  87+3  87+3  88+3  77+3  77+3  77+3  77+3  77+3   

     LB   LCB    CB   RCB    RB  Crossing  Finishing  HeadingAccuracy  \
0  59+2  47+2  47+2  47+2  59+2      84.0       95.0             70.0   
1  61+3  53+3  53+3  53+3  61+3      84.0       94.0             89.0   
2  60+3  47+3  47+3  47+3  60+3      79.0       87.0             62.0   
3     0     0     0     0     0      17.0       13.0             21.0   
4  73+3  66+3  66+3  66+3  73+3      93.0       82.0             55.0   

   ShortPassing  Volleys  Dribbling  Curve  FKAccuracy  LongPassing  \
0          90.0     86.0       97.0   93.0        94.0         87.0   
1          81.0     87.0       88.0   81.0        76.0         77.0   
2          84.0     84.0       96.0   88.0        87.0         78.0   
3          50.0     13.0       18.0   21.0        19.0         51.0   
4          92.0     82.0       86.0   85.0        83.0         91.0   

   BallControl  Acceleration  SprintSpeed  Agility  Reactions  Balance  \
0         96.0          91.0         86.0     91.0       95.0     95.0   
1         94.0          89.0         91.0     87.0       96.0     70.0   
2         95.0          94.0         90.0     96.0       94.0     84.0   
3         42.0          57.0         58.0     60.0       90.0     43.0   
4         91.0          78.0         76.0     79.0       91.0     77.0   

   ShotPower  Jumping  Stamina  Strength  LongShots  Aggression  \
0       85.0     68.0     72.0      59.0       94.0        48.0   
1       95.0     95.0     88.0      79.0       93.0        63.0   
2       80.0     61.0     81.0      49.0       82.0        56.0   
3       31.0     67.0     43.0      64.0       12.0        38.0   
4       91.0     63.0     90.0      75.0       91.0        76.0   

   Interceptions  Positioning  Vision  Penalties  Composure  Marking  \
0           22.0         94.0    94.0       75.0       96.0     33.0   
1           29.0         95.0    82.0       85.0       95.0     28.0   
2           36.0         89.0    87.0       81.0       94.0     27.0   
3           30.0         12.0    68.0       40.0       68.0     15.0   
4           61.0         87.0    94.0       79.0       88.0     6

In [26]:
df.loc[:,'Release Clause'] = df.loc[:,'Release Clause'].apply(value_to_float)
df.head()

Age Nationality  Overall  Potential                 Club        Value  \
0   31   Argentina       94         94         FC Barcelona  110500000.0   
1   33    Portugal       94         94             Juventus   77000000.0   
2   26      Brazil       92         93  Paris Saint-Germain  118500000.0   
3   27       Spain       91         93    Manchester United   72000000.0   
4   27     Belgium       91         92      Manchester City  102000000.0   

       Wage  Special  International Reputation  Skill Moves       Work Rate  \
0  565000.0     2202                       5.0          4.0  Medium/ Medium   
1  405000.0     2228                       5.0          5.0       High/ Low   
2  290000.0     2143                       5.0          5.0    High/ Medium   
3  260000.0     1471                       4.0          1.0  Medium/ Medium   
4  355000.0     2281                       4.0          4.0      High/ High   

  Real Face Position    LS    ST    RS    LW    LF    CF    RF    RW   LAM  \
0       Yes       RF  88+2  88+2  88+2  92+2  93+2  93+2  93+2  92+2  93+2   
1       Yes       ST  91+3  91+3  91+3  89+3  90+3  90+3  90+3  89+3  88+3   
2       Yes       LW  84+3  84+3  84+3  89+3  89+3  89+3  89+3  89+3  89+3   
3       Yes       GK     0     0     0     0     0     0     0     0     0   
4       Yes      RCM  82+3  82+3  82+3  87+3  87+3  87+3  87+3  87+3  88+3   

    CAM   RAM    LM   LCM    CM   RCM    RM   LWB   LDM   CDM   RDM   RWB  \
0  93+2  93+2  91+2  84+2  84+2  84+2  91+2  64+2  61+2  61+2  61+2  64+2   
1  88+3  88+3  88+3  81+3  81+3  81+3  88+3  65+3  61+3  61+3  61+3  65+3   
2  89+3  89+3  88+3  81+3  81+3  81+3  88+3  65+3  60+3  60+3  60+3  65+3   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4  88+3  88+3  88+3  87+3  87+3  87+3  88+3  77+3  77+3  77+3  77+3  77+3   

     LB   LCB    CB   RCB    RB  Crossing  Finishing  HeadingAccuracy  \
0  59+2  47+2  47+2  47+2  59+2      84.0       95.0             70.0   
1  61+3  53+3  53+3  53+3  61+3      84.0       94.0             89.0   
2  60+3  47+3  47+3  47+3  60+3      79.0       87.0             62.0   
3     0     0     0     0     0      17.0       13.0             21.0   
4  73+3  66+3  66+3  66+3  73+3      93.0       82.0             55.0   

   ShortPassing  Volleys  Dribbling  Curve  FKAccuracy  LongPassing  \
0          90.0     86.0       97.0   93.0        94.0         87.0   
1          81.0     87.0       88.0   81.0        76.0         77.0   
2          84.0     84.0       96.0   88.0        87.0         78.0   
3          50.0     13.0       18.0   21.0        19.0         51.0   
4          92.0     82.0       86.0   85.0        83.0         91.0   

   BallControl  Acceleration  SprintSpeed  Agility  Reactions  Balance  \
0         96.0          91.0         86.0     91.0       95.0     95.0   
1         94.0          89.0         91.0     87.0       96.0     70.0   
2         95.0          94.0         90.0     96.0       94.0     84.0   
3         42.0          57.0         58.0     60.0       90.0     43.0   
4         91.0          78.0         76.0     79.0       91.0     77.0   

   ShotPower  Jumping  Stamina  Strength  LongShots  Aggression  \
0       85.0     68.0     72.0      59.0       94.0        48.0   
1       95.0     95.0     88.0      79.0       93.0        63.0   
2       80.0     61.0     81.0      49.0       82.0        56.0   
3       31.0     67.0     43.0      64.0       12.0        38.0   
4       91.0     63.0     90.0      75.0       91.0        76.0   

   Interceptions  Positioning  Vision  Penalties  Composure  Marking  \
0           22.0         94.0    94.0       75.0       96.0     33.0   
1           29.0         95.0    82.0       85.0       95.0     28.0   
2           36.0         89.0    87.0       81.0       94.0     27.0   
3           30.0         12.0    68.0       40.0       68.0     15.0   
4           61.0         87.0    94.0       79.0       88.0     6

In [27]:
def modificarPuntosPosiciones(x):
    if x == 0:
        return 0
    else:
        lista = x.split("+")
        array = np.array(lista)
        return array[0]
        

In [28]:
columnNames = ["LS","ST","RS","LW","LF","CF","RF","RW","LAM","CAM","RAM","LM","LCM","CM","RCM","RM","LWB","LDM","CDM","RDM","RWB","LB","LCB","CB","RCB","RB"]

for columnName in columnNames:
    df[columnName] = df[columnName].apply(modificarPuntosPosiciones)
    
df.head()

Age Nationality  Overall  Potential                 Club        Value  \
0   31   Argentina       94         94         FC Barcelona  110500000.0   
1   33    Portugal       94         94             Juventus   77000000.0   
2   26      Brazil       92         93  Paris Saint-Germain  118500000.0   
3   27       Spain       91         93    Manchester United   72000000.0   
4   27     Belgium       91         92      Manchester City  102000000.0   

       Wage  Special  International Reputation  Skill Moves       Work Rate  \
0  565000.0     2202                       5.0          4.0  Medium/ Medium   
1  405000.0     2228                       5.0          5.0       High/ Low   
2  290000.0     2143                       5.0          5.0    High/ Medium   
3  260000.0     1471                       4.0          1.0  Medium/ Medium   
4  355000.0     2281                       4.0          4.0      High/ High   

  Real Face Position  LS  ST  RS  LW  LF  CF  RF  RW LAM CAM RAM  LM LCM  CM  \
0       Yes       RF  88  88  88  92  93  93  93  92  93  93  93  91  84  84   
1       Yes       ST  91  91  91  89  90  90  90  89  88  88  88  88  81  81   
2       Yes       LW  84  84  84  89  89  89  89  89  89  89  89  88  81  81   
3       Yes       GK   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
4       Yes      RCM  82  82  82  87  87  87  87  87  88  88  88  88  87  87   

  RCM  RM LWB LDM CDM RDM RWB  LB LCB  CB RCB  RB  Crossing  Finishing  \
0  84  91  64  61  61  61  64  59  47  47  47  59      84.0       95.0   
1  81  88  65  61  61  61  65  61  53  53  53  61      84.0       94.0   
2  81  88  65  60  60  60  65  60  47  47  47  60      79.0       87.0   
3   0   0   0   0   0   0   0   0   0   0   0   0      17.0       13.0   
4  87  88  77  77  77  77  77  73  66  66  66  73      93.0       82.0   

   HeadingAccuracy  ShortPassing  Volleys  Dribbling  Curve  FKAccuracy  \
0             70.0          90.0     86.0       97.0   93.0        94.0   
1             89.0          81.0     87.0       88.0   81.0        76.0   
2             62.0          84.0     84.0       96.0   88.0        87.0   
3             21.0          50.0     13.0       18.0   21.0        19.0   
4             55.0          92.0     82.0       86.0   85.0        83.0   

   LongPassing  BallControl  Acceleration  SprintSpeed  Agility  Reactions  \
0         87.0         96.0          91.0         86.0     91.0       95.0   
1         77.0         94.0          89.0         91.0     87.0       96.0   
2         78.0         95.0          94.0         90.0     96.0       94.0   
3         51.0         42.0          57.0         58.0     60.0       90.0   
4         91.0         91.0          78.0         76.0     79.0       91.0   

   Balance  ShotPower  Jumping  Stamina  Strength  LongShots  Aggression  \
0     95.0       85.0     68.0     72.0      59.0       94.0        48.0   
1     70.0       95.0     95.0     88.0      79.0       93.0        63.0   
2     84.0       80.0     61.0     81.0      49.0       82.0        56.0   
3     43.0       31.0     67.0     43.0      64.0       12.0        38.0   
4     77.0       91.0     63.0     90.0      75.0       91.0        76.0   

   Interceptions  Positioning  Vision  Penalties  Composure  Marking  \
0           22.0         94.0    94.0       75.0       96.0     33.0   
1           29.0         95.0    82.0       85.0       95.0     28.0   
2           36.0         89.0    87.0       81.0       94.0     27.0   
3           30.0         12.0    68.0       40.0       68.0     15.0   
4           61.0         87.0    94.0       79.0       88.0     68.0   

   StandingTackle  SlidingTackle  Release Clause  
0            28.0           26.0     226500000.0  
1            31.0           23.0     127100000.0  
2            24.0           33.0     228100000.0  
3            21.0           13.0     138600000.0  
4            58.0           51.0     196400000.0

# Vamos a modificar columnas que hay que convertir a formato numérico para que el algoritmo pueda utilizarlos:

    - 'Real Face'
    - 'Club'
    - 'Position'
    - 'Nationality'
    - 'Work Rate'

Encuentro que el campo realFace puede ser algo relevante porque notamos cierta tendencia a que los jugadores menos baratos, son evidentemente menos populares, y por tanto en el FIFA a muchos no se les aplica la cara real al modelo3D porque eso lleva un trabajo extra y no sale tan rentable hacerlo para estos jugadores menos populares.

In [29]:
realFace = df.pop("Real Face")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(realFace, prefix='realFace').reset_index(drop=True)], axis=1, sort=False)
df.head()

Age Nationality  Overall  Potential                 Club        Value  \
0   31   Argentina       94         94         FC Barcelona  110500000.0   
1   33    Portugal       94         94             Juventus   77000000.0   
2   26      Brazil       92         93  Paris Saint-Germain  118500000.0   
3   27       Spain       91         93    Manchester United   72000000.0   
4   27     Belgium       91         92      Manchester City  102000000.0   

       Wage  Special  International Reputation  Skill Moves       Work Rate  \
0  565000.0     2202                       5.0          4.0  Medium/ Medium   
1  405000.0     2228                       5.0          5.0       High/ Low   
2  290000.0     2143                       5.0          5.0    High/ Medium   
3  260000.0     1471                       4.0          1.0  Medium/ Medium   
4  355000.0     2281                       4.0          4.0      High/ High   

  Position  LS  ST  RS  LW  LF  CF  RF  RW LAM CAM RAM  LM LCM  CM RCM  RM  \
0       RF  88  88  88  92  93  93  93  92  93  93  93  91  84  84  84  91   
1       ST  91  91  91  89  90  90  90  89  88  88  88  88  81  81  81  88   
2       LW  84  84  84  89  89  89  89  89  89  89  89  88  81  81  81  88   
3       GK   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
4      RCM  82  82  82  87  87  87  87  87  88  88  88  88  87  87  87  88   

  LWB LDM CDM RDM RWB  LB LCB  CB RCB  RB  Crossing  Finishing  \
0  64  61  61  61  64  59  47  47  47  59      84.0       95.0   
1  65  61  61  61  65  61  53  53  53  61      84.0       94.0   
2  65  60  60  60  65  60  47  47  47  60      79.0       87.0   
3   0   0   0   0   0   0   0   0   0   0      17.0       13.0   
4  77  77  77  77  77  73  66  66  66  73      93.0       82.0   

   HeadingAccuracy  ShortPassing  Volleys  Dribbling  Curve  FKAccuracy  \
0             70.0          90.0     86.0       97.0   93.0        94.0   
1             89.0          81.0     87.0       88.0   81.0        76.0   
2             62.0          84.0     84.0       96.0   88.0        87.0   
3             21.0          50.0     13.0       18.0   21.0        19.0   
4             55.0          92.0     82.0       86.0   85.0        83.0   

   LongPassing  BallControl  Acceleration  SprintSpeed  Agility  Reactions  \
0         87.0         96.0          91.0         86.0     91.0       95.0   
1         77.0         94.0          89.0         91.0     87.0       96.0   
2         78.0         95.0          94.0         90.0     96.0       94.0   
3         51.0         42.0          57.0         58.0     60.0       90.0   
4         91.0         91.0          78.0         76.0     79.0       91.0   

   Balance  ShotPower  Jumping  Stamina  Strength  LongShots  Aggression  \
0     95.0       85.0     68.0     72.0      59.0       94.0        48.0   
1     70.0       95.0     95.0     88.0      79.0       93.0        63.0   
2     84.0       80.0     61.0     81.0      49.0       82.0        56.0   
3     43.0       31.0     67.0     43.0      64.0       12.0        38.0   
4     77.0       91.0     63.0     90.0      75.0       91.0        76.0   

   Interceptions  Positioning  Vision  Penalties  Composure  Marking  \
0           22.0         94.0    94.0       75.0       96.0     33.0   
1           29.0         95.0    82.0       85.0       95.0     28.0   
2           36.0         89.0    87.0       81.0       94.0     27.0   
3           30.0         12.0    68.0       40.0       68.0     15.0   
4           61.0         87.0    94.0       79.0       88.0     68.0   

   StandingTackle  SlidingTackle  Release Clause  realFace_No  realFace_Yes  
0            28.0           26.0     226500000.0            0             1  
1            31.0           23.0     127100000.0            0             1  
2            24.0           33.0     228100000.0            0             1  
3            21.0           13.0     138600000.0            0             1  
4    

Antes hemos eliminado los jugadores cuyo club es desconocido. Pero aún debemos convertir el club a un valor numérico para que el algoritmo pueda tratarlo:

In [30]:
clb = df.pop("Club")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(clb, prefix='clb').reset_index(drop=True)], axis=1, sort=False)
df.head()

Age Nationality  Overall  Potential        Value      Wage  Special  \
0   31   Argentina       94         94  110500000.0  565000.0     2202   
1   33    Portugal       94         94   77000000.0  405000.0     2228   
2   26      Brazil       92         93  118500000.0  290000.0     2143   
3   27       Spain       91         93   72000000.0  260000.0     1471   
4   27     Belgium       91         92  102000000.0  355000.0     2281   

   International Reputation  Skill Moves       Work Rate Position  LS  ST  RS  \
0                       5.0          4.0  Medium/ Medium       RF  88  88  88   
1                       5.0          5.0       High/ Low       ST  91  91  91   
2                       5.0          5.0    High/ Medium       LW  84  84  84   
3                       4.0          1.0  Medium/ Medium       GK   0   0   0   
4                       4.0          4.0      High/ High      RCM  82  82  82   

   LW  LF  CF  RF  RW LAM CAM RAM  LM LCM  CM RCM  RM LWB LDM CDM RDM RWB  LB  \
0  92  93  93  93  92  93  93  93  91  84  84  84  91  64  61  61  61  64  59   
1  89  90  90  90  89  88  88  88  88  81  81  81  88  65  61  61  61  65  61   
2  89  89  89  89  89  89  89  89  88  81  81  81  88  65  60  60  60  65  60   
3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
4  87  87  87  87  87  88  88  88  88  87  87  87  88  77  77  77  77  77  73   

  LCB  CB RCB  RB  Crossing  Finishing  HeadingAccuracy  ShortPassing  \
0  47  47  47  59      84.0       95.0             70.0          90.0   
1  53  53  53  61      84.0       94.0             89.0          81.0   
2  47  47  47  60      79.0       87.0             62.0          84.0   
3   0   0   0   0      17.0       13.0             21.0          50.0   
4  66  66  66  73      93.0       82.0             55.0          92.0   

   Volleys  Dribbling  Curve  FKAccuracy  LongPassing  BallControl  \
0     86.0       97.0   93.0        94.0         87.0         96.0   
1     87.0       88.0   81.0        76.0         77.0         94.0   
2     84.0       96.0   88.0        87.0         78.0         95.0   
3     13.0       18.0   21.0        19.0         51.0         42.0   
4     82.0       86.0   85.0        83.0         91.0         91.0   

   Acceleration  SprintSpeed  Agility  Reactions  Balance  ShotPower  Jumping  \
0          91.0         86.0     91.0       95.0     95.0       85.0     68.0   
1          89.0         91.0     87.0       96.0     70.0       95.0     95.0   
2          94.0         90.0     96.0       94.0     84.0       80.0     61.0   
3          57.0         58.0     60.0       90.0     43.0       31.0     67.0   
4          78.0         76.0     79.0       91.0     77.0       91.0     63.0   

   Stamina  Strength  LongShots  Aggression  Interceptions  Positioning  \
0     72.0      59.0       94.0        48.0           22.0         94.0   
1     88.0      79.0       93.0        63.0           29.0         95.0   
2     81.0      49.0       82.0        56.0           36.0         89.0   
3     43.0      64.0       12.0        38.0           30.0         12.0   
4     90.0      75.0       91.0        76.0           61.0         87.0   

   Vision  Penalties  Composure  Marking  StandingTackle  SlidingTackle  \
0    94.0       75.0       96.0     33.0            28.0           26.0   
1    82.0       85.0       95.0     28.0            31.0           23.0   
2    87.0       81.0       94.0     27.0            24.0           33.0   
3    68.0       40.0       68.0     15.0            21.0           13.0   
4    94.0       79.0       88.0     68.0            58.0           51.0   

   Release Clause  realFace_No  realFace_Yes  clb_ SSV Jahn Regensburg  \
0     226500000.0            0             1                         0   
1     127100000.0            0             1                         0   
2     228100000.0            0             1                         0   
3     138600000.0            0         

Ahora convertimos la posicion principal de los jugadores a valor numérico:

In [31]:
pos = df.pop("Position")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(pos, prefix='pos').reset_index(drop=True)], axis=1, sort=False)
df.head()

Age Nationality  Overall  Potential        Value      Wage  Special  \
0   31   Argentina       94         94  110500000.0  565000.0     2202   
1   33    Portugal       94         94   77000000.0  405000.0     2228   
2   26      Brazil       92         93  118500000.0  290000.0     2143   
3   27       Spain       91         93   72000000.0  260000.0     1471   
4   27     Belgium       91         92  102000000.0  355000.0     2281   

   International Reputation  Skill Moves       Work Rate  LS  ST  RS  LW  LF  \
0                       5.0          4.0  Medium/ Medium  88  88  88  92  93   
1                       5.0          5.0       High/ Low  91  91  91  89  90   
2                       5.0          5.0    High/ Medium  84  84  84  89  89   
3                       4.0          1.0  Medium/ Medium   0   0   0   0   0   
4                       4.0          4.0      High/ High  82  82  82  87  87   

   CF  RF  RW LAM CAM RAM  LM LCM  CM RCM  RM LWB LDM CDM RDM RWB  LB LCB  CB  \
0  93  93  92  93  93  93  91  84  84  84  91  64  61  61  61  64  59  47  47   
1  90  90  89  88  88  88  88  81  81  81  88  65  61  61  61  65  61  53  53   
2  89  89  89  89  89  89  88  81  81  81  88  65  60  60  60  65  60  47  47   
3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
4  87  87  87  88  88  88  88  87  87  87  88  77  77  77  77  77  73  66  66   

  RCB  RB  Crossing  Finishing  HeadingAccuracy  ShortPassing  Volleys  \
0  47  59      84.0       95.0             70.0          90.0     86.0   
1  53  61      84.0       94.0             89.0          81.0     87.0   
2  47  60      79.0       87.0             62.0          84.0     84.0   
3   0   0      17.0       13.0             21.0          50.0     13.0   
4  66  73      93.0       82.0             55.0          92.0     82.0   

   Dribbling  Curve  FKAccuracy  LongPassing  BallControl  Acceleration  \
0       97.0   93.0        94.0         87.0         96.0          91.0   
1       88.0   81.0        76.0         77.0         94.0          89.0   
2       96.0   88.0        87.0         78.0         95.0          94.0   
3       18.0   21.0        19.0         51.0         42.0          57.0   
4       86.0   85.0        83.0         91.0         91.0          78.0   

   SprintSpeed  Agility  Reactions  Balance  ShotPower  Jumping  Stamina  \
0         86.0     91.0       95.0     95.0       85.0     68.0     72.0   
1         91.0     87.0       96.0     70.0       95.0     95.0     88.0   
2         90.0     96.0       94.0     84.0       80.0     61.0     81.0   
3         58.0     60.0       90.0     43.0       31.0     67.0     43.0   
4         76.0     79.0       91.0     77.0       91.0     63.0     90.0   

   Strength  LongShots  Aggression  Interceptions  Positioning  Vision  \
0      59.0       94.0        48.0           22.0         94.0    94.0   
1      79.0       93.0        63.0           29.0         95.0    82.0   
2      49.0       82.0        56.0           36.0         89.0    87.0   
3      64.0       12.0        38.0           30.0         12.0    68.0   
4      75.0       91.0        76.0           61.0         87.0    94.0   

   Penalties  Composure  Marking  StandingTackle  SlidingTackle  \
0       75.0       96.0     33.0            28.0           26.0   
1       85.0       95.0     28.0            31.0           23.0   
2       81.0       94.0     27.0            24.0           33.0   
3       40.0       68.0     15.0            21.0           13.0   
4       79.0       88.0     68.0            58.0           51.0   

   Release Clause  realFace_No  realFace_Yes  clb_ SSV Jahn Regensburg  \
0     226500000.0            0             1                         0   
1     127100000.0            0             1                         0   
2     228100000.0            0             1                         0   
3     138600000.0            0             1                         0   
4     196400000.0  

Hacemos lo mismo con la columna 'Nationality':

In [32]:
nation = df.pop("Nationality")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(nation, prefix='nation').reset_index(drop=True)], axis=1, sort=False)
df.head()

Age  Overall  Potential        Value      Wage  Special  \
0   31       94         94  110500000.0  565000.0     2202   
1   33       94         94   77000000.0  405000.0     2228   
2   26       92         93  118500000.0  290000.0     2143   
3   27       91         93   72000000.0  260000.0     1471   
4   27       91         92  102000000.0  355000.0     2281   

   International Reputation  Skill Moves       Work Rate  LS  ST  RS  LW  LF  \
0                       5.0          4.0  Medium/ Medium  88  88  88  92  93   
1                       5.0          5.0       High/ Low  91  91  91  89  90   
2                       5.0          5.0    High/ Medium  84  84  84  89  89   
3                       4.0          1.0  Medium/ Medium   0   0   0   0   0   
4                       4.0          4.0      High/ High  82  82  82  87  87   

   CF  RF  RW LAM CAM RAM  LM LCM  CM RCM  RM LWB LDM CDM RDM RWB  LB LCB  CB  \
0  93  93  92  93  93  93  91  84  84  84  91  64  61  61  61  64  59  47  47   
1  90  90  89  88  88  88  88  81  81  81  88  65  61  61  61  65  61  53  53   
2  89  89  89  89  89  89  88  81  81  81  88  65  60  60  60  65  60  47  47   
3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
4  87  87  87  88  88  88  88  87  87  87  88  77  77  77  77  77  73  66  66   

  RCB  RB  Crossing  Finishing  HeadingAccuracy  ShortPassing  Volleys  \
0  47  59      84.0       95.0             70.0          90.0     86.0   
1  53  61      84.0       94.0             89.0          81.0     87.0   
2  47  60      79.0       87.0             62.0          84.0     84.0   
3   0   0      17.0       13.0             21.0          50.0     13.0   
4  66  73      93.0       82.0             55.0          92.0     82.0   

   Dribbling  Curve  FKAccuracy  LongPassing  BallControl  Acceleration  \
0       97.0   93.0        94.0         87.0         96.0          91.0   
1       88.0   81.0        76.0         77.0         94.0          89.0   
2       96.0   88.0        87.0         78.0         95.0          94.0   
3       18.0   21.0        19.0         51.0         42.0          57.0   
4       86.0   85.0        83.0         91.0         91.0          78.0   

   SprintSpeed  Agility  Reactions  Balance  ShotPower  Jumping  Stamina  \
0         86.0     91.0       95.0     95.0       85.0     68.0     72.0   
1         91.0     87.0       96.0     70.0       95.0     95.0     88.0   
2         90.0     96.0       94.0     84.0       80.0     61.0     81.0   
3         58.0     60.0       90.0     43.0       31.0     67.0     43.0   
4         76.0     79.0       91.0     77.0       91.0     63.0     90.0   

   Strength  LongShots  Aggression  Interceptions  Positioning  Vision  \
0      59.0       94.0        48.0           22.0         94.0    94.0   
1      79.0       93.0        63.0           29.0         95.0    82.0   
2      49.0       82.0        56.0           36.0         89.0    87.0   
3      64.0       12.0        38.0           30.0         12.0    68.0   
4      75.0       91.0        76.0           61.0         87.0    94.0   

   Penalties  Composure  Marking  StandingTackle  SlidingTackle  \
0       75.0       96.0     33.0            28.0           26.0   
1       85.0       95.0     28.0            31.0           23.0   
2       81.0       94.0     27.0            24.0           33.0   
3       40.0       68.0     15.0            21.0           13.0   
4       79.0       88.0     68.0            58.0           51.0   

   Release Clause  realFace_No  realFace_Yes  clb_ SSV Jahn Regensburg  \
0     226500000.0            0             1                         0   
1     127100000.0            0             1                         0   
2     228100000.0            0             1                         0   
3     138600000.0            0             1                         0   
4     196400000.0            0             1                         0   

   clb_1. FC Hei

Y por último con 'Work Rate':

In [33]:
workRate = df.pop("Work Rate")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(workRate, prefix='workRate').reset_index(drop=True)], axis=1, sort=False)
df.head()

Age  Overall  Potential        Value      Wage  Special  \
0   31       94         94  110500000.0  565000.0     2202   
1   33       94         94   77000000.0  405000.0     2228   
2   26       92         93  118500000.0  290000.0     2143   
3   27       91         93   72000000.0  260000.0     1471   
4   27       91         92  102000000.0  355000.0     2281   

   International Reputation  Skill Moves  LS  ST  RS  LW  LF  CF  RF  RW LAM  \
0                       5.0          4.0  88  88  88  92  93  93  93  92  93   
1                       5.0          5.0  91  91  91  89  90  90  90  89  88   
2                       5.0          5.0  84  84  84  89  89  89  89  89  89   
3                       4.0          1.0   0   0   0   0   0   0   0   0   0   
4                       4.0          4.0  82  82  82  87  87  87  87  87  88   

  CAM RAM  LM LCM  CM RCM  RM LWB LDM CDM RDM RWB  LB LCB  CB RCB  RB  \
0  93  93  91  84  84  84  91  64  61  61  61  64  59  47  47  47  59   
1  88  88  88  81  81  81  88  65  61  61  61  65  61  53  53  53  61   
2  89  89  88  81  81  81  88  65  60  60  60  65  60  47  47  47  60   
3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
4  88  88  88  87  87  87  88  77  77  77  77  77  73  66  66  66  73   

   Crossing  Finishing  HeadingAccuracy  ShortPassing  Volleys  Dribbling  \
0      84.0       95.0             70.0          90.0     86.0       97.0   
1      84.0       94.0             89.0          81.0     87.0       88.0   
2      79.0       87.0             62.0          84.0     84.0       96.0   
3      17.0       13.0             21.0          50.0     13.0       18.0   
4      93.0       82.0             55.0          92.0     82.0       86.0   

   Curve  FKAccuracy  LongPassing  BallControl  Acceleration  SprintSpeed  \
0   93.0        94.0         87.0         96.0          91.0         86.0   
1   81.0        76.0         77.0         94.0          89.0         91.0   
2   88.0        87.0         78.0         95.0          94.0         90.0   
3   21.0        19.0         51.0         42.0          57.0         58.0   
4   85.0        83.0         91.0         91.0          78.0         76.0   

   Agility  Reactions  Balance  ShotPower  Jumping  Stamina  Strength  \
0     91.0       95.0     95.0       85.0     68.0     72.0      59.0   
1     87.0       96.0     70.0       95.0     95.0     88.0      79.0   
2     96.0       94.0     84.0       80.0     61.0     81.0      49.0   
3     60.0       90.0     43.0       31.0     67.0     43.0      64.0   
4     79.0       91.0     77.0       91.0     63.0     90.0      75.0   

   LongShots  Aggression  Interceptions  Positioning  Vision  Penalties  \
0       94.0        48.0           22.0         94.0    94.0       75.0   
1       93.0        63.0           29.0         95.0    82.0       85.0   
2       82.0        56.0           36.0         89.0    87.0       81.0   
3       12.0        38.0           30.0         12.0    68.0       40.0   
4       91.0        76.0           61.0         87.0    94.0       79.0   

   Composure  Marking  StandingTackle  SlidingTackle  Release Clause  \
0       96.0     33.0            28.0           26.0     226500000.0   
1       95.0     28.0            31.0           23.0     127100000.0   
2       94.0     27.0            24.0           33.0     228100000.0   
3       68.0     15.0            21.0           13.0     138600000.0   
4       88.0     68.0            58.0           51.0     196400000.0   

   realFace_No  realFace_Yes  clb_ SSV Jahn Regensburg  \
0            0             1                         0   
1            0             1                         0   
2            0             1                         0   
3            0             1                         0   
4            0             1                         0   

   clb_1. FC Heidenheim 1846  clb_1. FC Kaiserslautern  clb_1. FC Köln  \
0                          0          

# Normalización de datos

Convertimos todos los valores al tipo de valor numérico float:

In [34]:
list = df.columns.values
for column in list:
    df.loc[:, column] = df.loc[:,column].apply(lambda x: float(x))
df.head()    

Age  Overall  Potential        Value      Wage  Special  \
0  31.0     94.0       94.0  110500000.0  565000.0   2202.0   
1  33.0     94.0       94.0   77000000.0  405000.0   2228.0   
2  26.0     92.0       93.0  118500000.0  290000.0   2143.0   
3  27.0     91.0       93.0   72000000.0  260000.0   1471.0   
4  27.0     91.0       92.0  102000000.0  355000.0   2281.0   

   International Reputation  Skill Moves    LS    ST    RS    LW    LF    CF  \
0                       5.0          4.0  88.0  88.0  88.0  92.0  93.0  93.0   
1                       5.0          5.0  91.0  91.0  91.0  89.0  90.0  90.0   
2                       5.0          5.0  84.0  84.0  84.0  89.0  89.0  89.0   
3                       4.0          1.0   0.0   0.0   0.0   0.0   0.0   0.0   
4                       4.0          4.0  82.0  82.0  82.0  87.0  87.0  87.0   

     RF    RW   LAM   CAM   RAM    LM   LCM    CM   RCM    RM   LWB   LDM  \
0  93.0  92.0  93.0  93.0  93.0  91.0  84.0  84.0  84.0  91.0  64.0  61.0   
1  90.0  89.0  88.0  88.0  88.0  88.0  81.0  81.0  81.0  88.0  65.0  61.0   
2  89.0  89.0  89.0  89.0  89.0  88.0  81.0  81.0  81.0  88.0  65.0  60.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4  87.0  87.0  88.0  88.0  88.0  88.0  87.0  87.0  87.0  88.0  77.0  77.0   

    CDM   RDM   RWB    LB   LCB    CB   RCB    RB  Crossing  Finishing  \
0  61.0  61.0  64.0  59.0  47.0  47.0  47.0  59.0      84.0       95.0   
1  61.0  61.0  65.0  61.0  53.0  53.0  53.0  61.0      84.0       94.0   
2  60.0  60.0  65.0  60.0  47.0  47.0  47.0  60.0      79.0       87.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0      17.0       13.0   
4  77.0  77.0  77.0  73.0  66.0  66.0  66.0  73.0      93.0       82.0   

   HeadingAccuracy  ShortPassing  Volleys  Dribbling  Curve  FKAccuracy  \
0             70.0          90.0     86.0       97.0   93.0        94.0   
1             89.0          81.0     87.0       88.0   81.0        76.0   
2             62.0          84.0     84.0       96.0   88.0        87.0   
3             21.0          50.0     13.0       18.0   21.0        19.0   
4             55.0          92.0     82.0       86.0   85.0        83.0   

   LongPassing  BallControl  Acceleration  SprintSpeed  Agility  Reactions  \
0         87.0         96.0          91.0         86.0     91.0       95.0   
1         77.0         94.0          89.0         91.0     87.0       96.0   
2         78.0         95.0          94.0         90.0     96.0       94.0   
3         51.0         42.0          57.0         58.0     60.0       90.0   
4         91.0         91.0          78.0         76.0     79.0       91.0   

   Balance  ShotPower  Jumping  Stamina  Strength  LongShots  Aggression  \
0     95.0       85.0     68.0     72.0      59.0       94.0        48.0   
1     70.0       95.0     95.0     88.0      79.0       93.0        63.0   
2     84.0       80.0     61.0     81.0      49.0       82.0        56.0   
3     43.0       31.0     67.0     43.0      64.0       12.0        38.0   
4     77.0       91.0     63.0     90.0      75.0       91.0        76.0   

   Interceptions  Positioning  Vision  Penalties  Composure  Marking  \
0           22.0         94.0    94.0       75.0       96.0     33.0   
1           29.0         95.0    82.0       85.0       95.0     28.0   
2           36.0         89.0    87.0       81.0       94.0     27.0   
3           30.0         12.0    68.0       40.0       68.0     15.0   
4           61.0         87.0    94.0       79.0       88.0     68.0   

   StandingTackle  SlidingTackle  Release Clause  realFace_No  realFace_Yes  \
0            28.0           26.0     226500000.0          0.0           1.0   
1            31.0           23.0     127100000.0          0.0           1.0   
2            24.0           33.0     228100000.0          0.0           1.0   
3            21.0           13.0     138600000.0          0.0           1.0   
4            58.0           51.

Normalizamos los valores utilizando MinMaxScaler del paquete sklearn:

In [35]:
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(df) 
df.loc[:,:] = scaled_values
df.head()

Age   Overall  Potential     Value      Wage   Special  \
0  0.517241  1.000000   0.978723  0.932484  1.000000  0.910836   
1  0.586207  1.000000   0.978723  0.649759  0.716312  0.926935   
2  0.344828  0.958333   0.957447  1.000000  0.512411  0.874303   
3  0.379310  0.937500   0.957447  0.607562  0.459220  0.458204   
4  0.379310  0.937500   0.936170  0.860748  0.627660  0.959752   

   International Reputation  Skill Moves        LS        ST        RS  \
0                      1.00         0.75  0.967033  0.967033  0.967033   
1                      1.00         1.00  1.000000  1.000000  1.000000   
2                      1.00         1.00  0.923077  0.923077  0.923077   
3                      0.75         0.00  0.000000  0.000000  0.000000   
4                      0.75         0.75  0.901099  0.901099  0.901099   

         LW        LF        CF        RF        RW       LAM       CAM  \
0  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
1  0.967391  0.967742  0.967742  0.967742  0.967391  0.946237  0.946237   
2  0.967391  0.956989  0.956989  0.956989  0.967391  0.956989  0.956989   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.945652  0.935484  0.935484  0.935484  0.945652  0.946237  0.946237   

        RAM        LM       LCM        CM       RCM        RM       LWB  \
0  1.000000  1.000000  0.954545  0.954545  0.954545  1.000000  0.752941   
1  0.946237  0.967033  0.920455  0.920455  0.920455  0.967033  0.764706   
2  0.956989  0.967033  0.920455  0.920455  0.920455  0.967033  0.764706   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.946237  0.967033  0.988636  0.988636  0.988636  0.967033  0.905882   

        LDM       CDM       RDM       RWB        LB       LCB        CB  \
0  0.701149  0.701149  0.701149  0.752941  0.702381  0.540230  0.540230   
1  0.701149  0.701149  0.701149  0.764706  0.726190  0.609195  0.609195   
2  0.689655  0.689655  0.689655  0.764706  0.714286  0.540230  0.540230   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.885057  0.885057  0.885057  0.905882  0.869048  0.758621  0.758621   

        RCB        RB  Crossing  Finishing  HeadingAccuracy  ShortPassing  \
0  0.540230  0.702381  0.897727   1.000000         0.733333      0.965116   
1  0.609195  0.726190  0.897727   0.989247         0.944444      0.860465   
2  0.540230  0.714286  0.840909   0.913978         0.644444      0.895349   
3  0.000000  0.000000  0.136364   0.118280         0.188889      0.500000   
4  0.758621  0.869048  1.000000   0.860215         0.566667      0.988372   

    Volleys  Dribbling     Curve  FKAccuracy  LongPassing  BallControl  \
0  0.953488   1.000000  0.988636    1.000000     0.928571     1.000000   
1  0.965116   0.903226  0.852273    0.802198     0.809524     0.978022   
2  0.930233   0.989247  0.931818    0.923077     0.821429     0.989011   
3  0.104651   0.150538  0.170455    0.175824     0.500000     0.406593   
4  0.906977   0.881720  0.897727    0.879121     0.976190     0.945055   

   Acceleration  SprintSpeed   Agility  Reactions  Balance  ShotPower  \
0      0.929412     0.880952  0.939024   0.986667   0.9875   0.892473   
1      0.905882     0.940476  0.890244   1.000000   0.6750   1.000000   
2      0.964706     0.928571  1.000000   0.973333   0.8500   0.838710   
3      0.529412     0.547619  0.560976   0.920000   0.3375   0.311828   
4      0.776471     0.761905  0.792683   0.933333   0.7625   0.956989   

   Jumping   Stamina  Strength  LongShots  Aggression  Interceptions  \
0   0.6625  0.714286    0.5250   1.000000    0.440476       0.213483   
1   1.0000  0.904762    0.7750   0.989011    0.619048       0.292135   
2   0.5750  0.821429    0.4000   0.868132    0.535714       0.370787   
3   0.6500  0.369048    0.5875   0.098901    0.321429       0.303371   
4   0.6000  0.928571    0.7250   0.967033    0.773810       0.651685   

   Positioning    Vision  Penalti

# Objetivo de la normalización de datos en esta práctica:

El objetivo de la normalización es cambiar los valores de las columnas numéricas en el conjunto de datos a una escala común, sin distorsionar las diferencias en los rangos de valores. Para el aprendizaje automático, no todos los conjuntos de datos requieren normalización. Solo se requiere cuando las características tienen diferentes rangos.

Por ejemplo, en un conjunto de datos que contiene dos características, edad e ingresos, donde la edad varía de 0 a 100, mientras que los ingresos varían de 0 a 100.000 o más, los ingresos son unas 1.000 veces mayores que la edad. Entonces, estas dos características están en rangos muy diferentes. Cuando hacemos un análisis, como la regresión lineal multivariante, por ejemplo, el ingreso atribuido influirá más en el resultado debido a su mayor valor. Pero esto no significa necesariamente que sea más importante como predictor. Entonces normalizamos los datos para traer todas las variables al mismo rango.

# Predicción 

Guardamos el estado actual del dataFrame por si queremos volver a hacer la predicción con cambios, ya que al hacer la predicción
sacamos el campo Value de la tabla.

In [36]:
df2 = df.copy()

In [37]:
val = df.pop("Value")

Separamos los datos:

In [38]:
X_train, X_test, y_train, y_test = train_test_split(df, val, test_size=0.33, random_state=42)

In [39]:
len(X_train)

11150

Entrenamos un modelo de Regresión Lineal:

In [40]:
reg = linear_model.LinearRegression().fit(X_train, y_train)

Obtenemos una métrica 𝑅^2 para la regresión, utilizamos la implementación de sckikit-learn:

In [41]:
preds = reg.predict(X_test)

In [42]:
preds[0]

0.04016577824950218

In [43]:
y_test.iloc[0]

0.042113258502827246

In [44]:
r2_score(preds, y_test)

-0.001093487360577683

# Como podemos ver, en con el estado actual del DataFrame, aunque el valor predecido se aproxima en cierta manera al valor real, aún se puede mejorar mucho la puntuación.

# ¿Qué puede estar frenando la calidad de la aproximación?¿Como es posible que habiendo quitado todas las columnas que hemos visto y justificado que no sirven, aún así obtengamos una puntuación no muy buena?

# Lo primero que llama la atención es la cantidad de columnas creadas para que el algoritmo pueda tratar con los campos 'Club', 'Position', 'Nationality' y 'Work Rate'.

# ¿Quizás estos campos están "mareando" al algoritmo? ¿Realmente tienen una relación fuerte con el campo Value de manera que sirven para precedir el valor del campo Value de un jugador? Vamos a razonarlo:

<b>Club</b>: este es algo confuso para mí así que lo voy a dejar por si acaso. El hecho de que un jugador pertenezca a un determinado club quizás ayuda a determinar si su Value es alto. Por ejemplo el hecho de que un jugador pertenezca a un equipo tan dominante como el FC Barcelona o el Real Madrid C.F. seguramente significa que ese jugador es más caro que uno que pertenece por ejemplo al Levante.

<b>Position</b>: este campo indica la posición principal en la que juega un jugador. Realmente quizás no tiene una relación fuerte con el Value, ya que podemos tener jugadores tanto caros como baratos en cualquier posicion. Existen muchos defensas caros,porteros caro como De Gea, delanteros caros, etc. En resumen, hay jugadores caros y baratos muy distribuidos con todas las posiciones, no vemos nada del estilo que todos los delanteros sean caros, o algo parecido. Entonces vamos a probar de quitar esta columna.

<b>Nationality</b>: consiste en el país donde nació el jugador o su nacionalidad vigente actual. En un principio alguien podría pensar por ejemplo que quizás los jugadores brasileños son más caros porque existen varios casos de jugadores famosos brasileños muy buenos en la historia del fútbol. Pero realmente esto es solo una ilusión, tenemos un dataFrame con muchos jugadores de muchas nacionalidades. Tenemos jugadores caros y baratos de Alemania, Argentina, México, Holanda, etc. Están distribuidos. Este campo entonces concluimos que no es muy importante. Vamos a probar de quitarlo también.

<b>Work Rate</b>: indica el comportamiento del jugador en cuanto a la posición en el campo y el movimiendo y la frecuencia con que ataca y defiende. Esta dividido en Ataque y Defensa (<Ataque>/<Defensa>). Un jugador con "Low" en defensa por ejemplo, significa que se mantendrá en su posicion mientras el equipo ataca. La primera intuición que uno puede tener, yo incluido, es que High/High es lo mejor, pero esto no es así, ya que un jugador con esos valores estará fuera de la posicion conveniente a menudo, afectando a la organización del equipo. Además, en general el Work Rate deriva de la posición en que juega un jugador, que ya se ha razonado que no tiene una relación fuerte con el Value. Podemos ver jugadores Medium/Medium caros pero también los hay baratos. Creo que no vale la pena dejar esta columna. También la voy a quitar.


# Dicho esto, vamos a quitar las columnas que se crearon al aplicar get_dummies a 'Position', 'Nationality' y 'Work Rate' del data frame:

In [45]:
df2 = df2.iloc[:,:717]
df2.head()

Age   Overall  Potential     Value      Wage   Special  \
0  0.517241  1.000000   0.978723  0.932484  1.000000  0.910836   
1  0.586207  1.000000   0.978723  0.649759  0.716312  0.926935   
2  0.344828  0.958333   0.957447  1.000000  0.512411  0.874303   
3  0.379310  0.937500   0.957447  0.607562  0.459220  0.458204   
4  0.379310  0.937500   0.936170  0.860748  0.627660  0.959752   

   International Reputation  Skill Moves        LS        ST        RS  \
0                      1.00         0.75  0.967033  0.967033  0.967033   
1                      1.00         1.00  1.000000  1.000000  1.000000   
2                      1.00         1.00  0.923077  0.923077  0.923077   
3                      0.75         0.00  0.000000  0.000000  0.000000   
4                      0.75         0.75  0.901099  0.901099  0.901099   

         LW        LF        CF        RF        RW       LAM       CAM  \
0  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
1  0.967391  0.967742  0.967742  0.967742  0.967391  0.946237  0.946237   
2  0.967391  0.956989  0.956989  0.956989  0.967391  0.956989  0.956989   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.945652  0.935484  0.935484  0.935484  0.945652  0.946237  0.946237   

        RAM        LM       LCM        CM       RCM        RM       LWB  \
0  1.000000  1.000000  0.954545  0.954545  0.954545  1.000000  0.752941   
1  0.946237  0.967033  0.920455  0.920455  0.920455  0.967033  0.764706   
2  0.956989  0.967033  0.920455  0.920455  0.920455  0.967033  0.764706   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.946237  0.967033  0.988636  0.988636  0.988636  0.967033  0.905882   

        LDM       CDM       RDM       RWB        LB       LCB        CB  \
0  0.701149  0.701149  0.701149  0.752941  0.702381  0.540230  0.540230   
1  0.701149  0.701149  0.701149  0.764706  0.726190  0.609195  0.609195   
2  0.689655  0.689655  0.689655  0.764706  0.714286  0.540230  0.540230   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.885057  0.885057  0.885057  0.905882  0.869048  0.758621  0.758621   

        RCB        RB  Crossing  Finishing  HeadingAccuracy  ShortPassing  \
0  0.540230  0.702381  0.897727   1.000000         0.733333      0.965116   
1  0.609195  0.726190  0.897727   0.989247         0.944444      0.860465   
2  0.540230  0.714286  0.840909   0.913978         0.644444      0.895349   
3  0.000000  0.000000  0.136364   0.118280         0.188889      0.500000   
4  0.758621  0.869048  1.000000   0.860215         0.566667      0.988372   

    Volleys  Dribbling     Curve  FKAccuracy  LongPassing  BallControl  \
0  0.953488   1.000000  0.988636    1.000000     0.928571     1.000000   
1  0.965116   0.903226  0.852273    0.802198     0.809524     0.978022   
2  0.930233   0.989247  0.931818    0.923077     0.821429     0.989011   
3  0.104651   0.150538  0.170455    0.175824     0.500000     0.406593   
4  0.906977   0.881720  0.897727    0.879121     0.976190     0.945055   

   Acceleration  SprintSpeed   Agility  Reactions  Balance  ShotPower  \
0      0.929412     0.880952  0.939024   0.986667   0.9875   0.892473   
1      0.905882     0.940476  0.890244   1.000000   0.6750   1.000000   
2      0.964706     0.928571  1.000000   0.973333   0.8500   0.838710   
3      0.529412     0.547619  0.560976   0.920000   0.3375   0.311828   
4      0.776471     0.761905  0.792683   0.933333   0.7625   0.956989   

   Jumping   Stamina  Strength  LongShots  Aggression  Interceptions  \
0   0.6625  0.714286    0.5250   1.000000    0.440476       0.213483   
1   1.0000  0.904762    0.7750   0.989011    0.619048       0.292135   
2   0.5750  0.821429    0.4000   0.868132    0.535714       0.370787   
3   0.6500  0.369048    0.5875   0.098901    0.321429       0.303371   
4   0.6000  0.928571    0.7250   0.967033    0.773810       0.651685   

   Positioning    Vision  Penalti

# Volvemos a realizar la predicción:

In [46]:
val = df2.pop("Value")

In [47]:
X_train, X_test, y_train, y_test = train_test_split(df2, val, test_size=0.33, random_state=42)

In [48]:
len(X_train)

11150

In [49]:
reg = linear_model.LinearRegression().fit(X_train, y_train)

In [50]:
preds = reg.predict(X_test)

In [51]:
preds[0]

0.04050445556640625

In [52]:
y_test.iloc[0]

0.042113258502827246

In [53]:
r2_score(preds, y_test)

0.9935430737047646

# Interpretación del resultado

Como podemos ver, los campos 'Position', 'Nationality' y 'Work Rate' eran el principal problema.

Obtenemos ahora una puntuación de 0.99. ¿Cómo se interpreta esto?

R2_score no es un porcentaje. Lo que hace esta función es computar el coeficiente de determinación. El rango de puntuación es [-1,1].

Más concretamente, r2_score representa la proporción de varianza de 'y' que ha sido determinada por las variables inde
pendientes del modelo. Aporta una indicación de precisión de ajuste, y por lo tanto, una medida de qué tan problabe es
que el modelo prediga muestras que no ha visto, a través de la proporción de la varianza determinada.

Por lo tanto, con una puntuación de 0.99, podemos saber que nuestro algoritmo predecirá con mucha precesión el valor
de Value a través de las otras variables del data set.

# ---------------------------------------------------------------------------------------------------------------

# Conclusiones

Con la realización de esta práctica, se puede ver que se ha conseguido realizar una buena predicción con un algoritmo de aprendizaje gracias a un análisis y limpieza de datos aplicando la lógica, sin necesidad de aplicar conceptos muy técnicos como por ejemplo la correlación de Pearson o cosas parecidas.

Hay que destacar el impacto de realizar get_dummies a una columna. Se crean muchisimas columnas que llenan el dataFrame de muchos valores, y a parte de suponer una mayor carga para el ordenador, pueden cambiar completamente la precisión del algoritmo de aprendizaje si no se ha comprobado que el uso de estos campos es adecuado para el objetivo.